In [1]:
DATAPATH = "results"

# Installs and imports

In [2]:
#!pip install pingouin > /dev/null

In [3]:
import pingouin as pg
import pandas as pd
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, f1_score
from sklearn import metrics
import pickle
import os
from tqdm import tqdm
import gzip
import scipy.stats as stats
import itertools
from statsmodels.stats.anova import AnovaRM
from collections import Counter

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

/home/kali/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/kali/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Data loading

In [4]:
label_names = ["human", "machine"] #0, 1
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {v:k for k,v in id2label.items()}

In [5]:
multidomain = pd.read_csv(f"{DATAPATH.replace('results', 'dataset')}/multidomain.csv.gz")

In [6]:
temp = multidomain[(multidomain.split == 'test') & (multidomain.domain == 'social_media')]
temp['source'] = temp['source'].str.replace('multisocial_', '')
temp.groupby(['source'])['length'].describe()

/tmp/ipykernel_634/3072383140.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['source'] = temp['source'].str.replace('multisocial_', '')


,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
discord,27652.0,16.213764,26.201387,3.0,5.0,9.0,15.0,197.0
gab,30740.0,25.979538,30.708710,3.0,9.0,16.0,30.0,200.0
telegram,44385.0,23.820390,32.579460,3.0,7.0,12.0,24.0,200.0
twitter,25172.0,25.470563,26.693076,3.0,11.0,18.0,30.0,200.0
whatsapp,10878.0,30.622909,33.921798,3.0,9.0,18.0,38.0,200.0


In [7]:
test_results = []

In [8]:

files = glob.glob(DATAPATH + "/pretrained/*.csv.gz")
for f in tqdm(files, total= len(files)):
   df = pd.read_csv(f)
   df = pd.concat([multidomain, df], axis=1)
   df['label'] = ['human' if 'human' in x else 'machine' for x in df['multi_label']]
   df['predictions'] = ['human' if (('human' in str(x)) or ('0' in str(x))) else 'machine' for x in df['predictions']]
   df['Category'] = 'P'

   if ('ruroberta' in f.lower()):
     df['prediction_probs'] = 1 - df['prediction_probs']
   test_results.append({f.split('/')[-1].replace('multidomain.csv_', '').replace('.csv.gz', '') : df})

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.11it/s]


In [9]:

files = glob.glob(DATAPATH + "/statistical/*.csv.gz")
for f in tqdm(files, total= len(files)):
   df = pd.read_csv(f)
   df = pd.concat([multidomain, df], axis=1)
   df['label'] = ['human' if 'human' in x else 'machine' for x in df['multi_label']]
   df['predictions'] = ['human' if (('human' in str(x)) or ('0' in str(x))) else 'machine' for x in df['predictions']]
   df['Category'] = 'S'

   if ('llr' in f.lower()):
     continue
     df['prediction_probs'] = 1 - df['prediction_probs']
   if ('s5' in f.lower()):
     if ('gpt-j' not in f.lower()): continue #use only GPT-J based S5 metric
     df['ll'] = 1 - df['ll']
     temp = pd.DataFrame()
     temp[['ll', 'entropy', 'rank', 'log-rank', 'llm_deviation']] = 1 / df[['ll', 'entropy', 'rank', 'log-rank', 'llm_deviation']]
     temp = temp.fillna(0.0)
     df['prediction_probs'] = temp.prod(axis=1).fillna(0.0).replace(np.inf, 0.0)
     
     #llm_deviation
     temp = df.copy()
     temp['prediction_probs'] = 1 - temp['llm_deviation']
     test_results.append({f.split('/')[-1].replace('multidomain.csv_', '').replace('.csv.gz', '').replace('s5', 'llm_deviation') : temp})

     #lrr
     temp = df.copy()
     temp['prediction_probs'] = temp['ll'] / temp['log-rank']
     temp['prediction_probs'] = temp['prediction_probs'].fillna(0.0).replace(np.inf, 0.0)
     test_results.append({f.split('/')[-1].replace('multidomain.csv_', '').replace('.csv.gz', '').replace('s5', 'lrr') : temp})
   test_results.append({f.split('/')[-1].replace('multidomain.csv_', '').replace('.csv.gz', '') : df})

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.70s/it]


In [10]:

files = glob.glob(DATAPATH + "/finetuned/*.csv.gz")
for f in tqdm(files, total= len(files)):
   #print(f)
   df = pd.read_csv(f)
   if 'prediction_probs' not in df.columns:
    df.columns.values[-1] = 'prediction_probs'
   if 'predictions' not in df.columns: df['predictions'] = 0
   df = pd.concat([multidomain, df[['predictions', 'prediction_probs']]], axis=1)
   df['prediction_probs'] = df['prediction_probs'].fillna(0.0).replace(np.inf, 0.0)
   df['label'] = ['human' if 'human' in x else 'machine' for x in df['multi_label']]
   df['predictions'] = ['human' if (('human' in str(x)) or ('0' in str(x))) else 'machine' for x in df['predictions']]
   df['Category'] = 'F'
   test_results.append({f.split('/')[-1].replace('multidomain.csv_', '').replace('.csv.gz', '') : df})

100%|███████████████████████████████████████████████████████████████████████████████████| 73/73 [00:56<00:00,  1.29it/s]


# Results analysis

## Definitions

In [11]:
#https://github.com/scikit-learn/scikit-learn/issues/26808
def report_np(y_true, y_pred, n_classes):

    classes = np.arange(n_classes)[None, :]
    supp = classes == y_true[:, None]
    tmp = classes == y_pred[:, None]
    hits = (tmp & supp).sum(axis=0)
    pred = tmp.sum(axis=0)
    n = y_true.shape[0]

    supp = supp.sum(axis=0)
    #https://stackoverflow.com/questions/26248654/how-to-return-0-with-divide-by-zero
    #prec = hits / pred
    pred_inv = np.array([1/i if i!=0 else 0 for i in pred])
    prec = hits * pred_inv
    #prec = np.divide(hits, pred, out=np.zeros(hits.shape, dtype=float), where=pred!=0)
    #rec = hits / supp
    supp_inv = np.array([1/i if i!=0 else 0 for i in supp])
    rec = hits * supp_inv
    #rec = np.divide(hits, supp, out=np.zeros(hits.shape, dtype=float), where=supp!=0)
    balanced_acc = rec.mean()
    prec_rec = prec + rec
    prec_rec_mult = 2 * prec * rec
    #f1 = prec_rec_mult / prec_rec
    prec_rec_inv = np.array([1/i if i!=0 else 0 for i in prec_rec])
    f1 = prec_rec_mult * prec_rec_inv
    #f1 = np.divide(prec_rec_mult, prec_rec, out=np.zeros(prec_rec_mult.shape, dtype=float), where=prec_rec!=0)
    acc = hits.sum() / n
    stacked = np.vstack([prec, rec, f1])
    macro = stacked.mean(axis=1)
    weighted = stacked @ supp / n

    return hits, pred - hits, acc, balanced_acc, supp, prec, rec, f1 , macro, weighted

def report_todict(hits, miss, acc, balanced_acc, supp, prec, rec, f1 , macro, weighted):
  report = {}
  TN = hits[0]
  FN = miss[0]
  TP = hits[1]
  FP = miss[1]
  report['fpr'] = FP/(FP+TN) if (FP+TN) > 0 else 0
  report['fnr'] = FN/(TP+FN) if (TP+FN) > 0 else 0
  human = {}
  human['precision'] = prec[0]
  human['recall'] = rec[0]
  human['f1-score'] = f1[0]
  human['support'] = supp[0]
  report['human'] = human
  machine = {}
  machine['precision'] = prec[1]
  machine['recall'] = rec[1]
  machine['f1-score'] = f1[1]
  machine['support'] = supp[1]
  report['machine'] = machine
  report['accuracy'] = acc
  report['balanced accuracy'] = balanced_acc
  macro_avg = {}
  macro_avg['precision'] = macro[0]
  macro_avg['recall'] = macro[1]
  macro_avg['f1-score'] = macro[2]
  report['macro avg'] = macro_avg
  weighted_avg = {}
  weighted_avg['precision'] = weighted[0]
  weighted_avg['recall'] = weighted[1]
  weighted_avg['f1-score'] = weighted[2]
  report['weighted avg'] = weighted_avg
  return report

In [12]:
def rename_detector(detector_name):
  detector_name = (detector_name
    .lower()
    .replace('meta-', '')
    .replace('orzhan-', '')
    .replace('nealcly-', '')
    .replace('hello-simpleai-', '')
    .replace('andreas122001-', '')
    .replace('openai-community-', '')
    .replace('_threshold', '')
    .replace('metric', '')
    .replace('gpt-j-6b_', '')
    .replace('s5', 'S5')
    .replace('llm_deviation', 'LLM-Deviation')
    .replace('lrr', 'DetectLLM-LRR')
    .replace('llr', 'DetectLLM-LRR')
    .replace('-first-social-media', '-MultiSocial')
    .replace('-first-news', '-MULTITuDE')
    .replace('xlm', 'XLM')
    .replace('roberta', 'RoBERTa')
    .replace('mdeberta', 'mDeBERTa')
    .replace('bert', 'BERT')
    .replace('bloomz', 'BLOOMZ')
    .replace('falcon', 'Falcon')
    .replace('mistral', 'Mistral')
    .replace('llama', 'Llama')
    .replace('aya', 'Aya')
    .replace('opt-iml-max', 'OPT-IML-Max')
    .replace('openai', 'OpenAI')
    .replace('detection-longformer', 'Longformer Detector')
    .replace('detect', 'Detect')
    .replace('fast', 'Fast')
    .replace('chatgpt', 'ChatGPT')
    #.replace('gpt-j', 'GPT-J')
    .replace('gpt', 'GPT')
    .replace('chinese', 'Chinese')
    .replace('longformer', 'Longformer')
    .replace('binoculars', 'Binoculars')
    )

  return detector_name

In [13]:
rename_generators = {'opt-iml-max-30b': 'OPT-IML-Max-30b', 'Mistral-7B-Instruct-v0.2': 'Mistral-7B-Instruct-v0.2', 'vicuna-13b': 'Vicuna-13b',
       'gpt-3.5-turbo-0125': 'GPT-3.5-Turbo-0125', 'aya-101': 'Aya-101', 'v5-Eagle-7B-HF': 'v5-Eagle-7B-HF', 'gemini': 'Gemini'}

In [14]:
rename_platforms = {'gab': 'Gab', 'whatsapp': 'WhatsApp', 'twitter': 'Twitter', 'telegram': 'Telegram', 'discord': 'Discord', '{all}': '{all}'}

In [15]:
#extract_platforms = {'-cross-platform-1-cut-majority': '_all', '-cross-platform-4-cut-majority': '_Gab', '-cross-platform-6-cut-majority': '_WhatsApp', '-cross-platform-3-cut-majority': '_Twitter', '-cross-platform-2-cut-majority': '_Telegram', '-cross-platform-5-cut-majority': '_Discord'}
def extract_platform(s):
  s = (s.replace('-cross-platform-1-cut-majority-one-fifth-train-size', '_all')
        .replace('-cross-platform-1-cut-majority', '_all-x5')
        .replace('-cross-platform-4-cut-majority', '_Gab')
        .replace('-cross-platform-6-cut-majority', '_WhatsApp')
        .replace('-cross-platform-3-cut-majority', '_Twitter')
        .replace('-cross-platform-2-cut-majority', '_Telegram')
        .replace('-cross-platform-5-cut-majority', '_Discord')
  )
  return s

In [16]:
def extract_language(s):
  s = (s.replace('-cross-language-1-cut-majority-one-third-train-size', '_{en-es-ru}')
        .replace('-cross-language-1-cut-majority', '_{en-es-ru-x3}')
        .replace('-cross-language-2-cut-majority', '_en')
        .replace('-cross-language-3-cut-majority', '_es')
        .replace('-cross-language-4-cut-majority', '_ru')
  )
  return s

In [17]:
to_category = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    to_category[model] = data.Category.unique()[0]

100%|███████████████████████████████████████████████████████████████████████████████████| 83/83 [00:01<00:00, 47.80it/s]


In [18]:
to_language_family = {
    'el': 'Greek',
    'ar': 'Semitic', 'zh': 'Sino-Tibetan',
    'ca': 'Romance', 'es': 'Romance', 'pt': 'Romance',
    'en': 'Germanic', 'ga': 'Celtic', 'gd': 'Celtic',
    'hu': 'Uralic', 'et': 'Uralic',
    'de': 'Germanic', 'nl': 'Germanic',
    'cs': 'Slavic-Latin', 'pl': 'Slavic-Latin', 'sk': 'Slavic-Latin',
    'hr': 'Slavic-Latin', 'sl': 'Slavic-Latin',
    'ro': 'Romance',
    'bg': 'Slavic-Cyrillic', 'uk': 'Slavic-Cyrillic',  'ru': 'Slavic-Cyrillic',
}

In [19]:
def highlight_categories(s):
    v = s['Category']
    color = 'background-color: #b6d7a8;' if v == 'F' else 'background-color: #f9cb9c;' if v == 'S' else 'background-color: #9fc5e8;'
    return [color for v in s]

In [20]:
def auc_roc_reliable(fpr, tpr, labels):
    res =  auc(fpr, tpr)
    counts = Counter(labels)
    return res if (len(counts) and counts[0] >= 10 and counts[1] >= 10) else pd.NA

## Benchmark Comparison

In [21]:

#social media test data - th calibration for macroF1 based on train data
auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    train = data[(data.split == 'train') & (data.domain == 'social_media') & (~data.multi_label.str.contains('gemini'))]
    temp = data[(data.split == 'test') & (data.domain == 'social_media')]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}

  train_labels = [label2id[x] for x in train['label']]
  fpr, tpr, thresholds = roc_curve(train_labels, train['prediction_probs'])
  predictions = [1 if ((x >= thresholds[fpr <= 0.05][-1])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_5'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
  predictions = [1 if ((x >= thresholds[np.argmax(tpr - fpr)])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_o'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': rename_detector(model), 'Category': to_category[model], 'AUC ROC': v['auc'], 'MacroF1@5%FPR': v['cr_5']['macro avg']['f1-score'], 'MacroF1@optim': v['cr_o']['macro avg']['f1-score']}, index=[0])
  results_all = pd.concat([results_all, temp])
results_all = results_all.sort_values(by=['AUC ROC'], ascending=False).reset_index(drop=True)
results_all.index = results_all.index + 1
results_all.style.format(na_rep=0, precision=4)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 465.59it/s]


,Detector,Category,AUC ROC,MacroF1@5%FPR,MacroF1@optim
1,Llama-3-8b-MultiSocial,F,0.9769,0.8696,0.8567
2,Mistral-7b-v0.1-MultiSocial,F,0.9768,0.8692,0.8619
3,Aya-101-MultiSocial,F,0.9731,0.8462,0.8457
4,Falcon-rw-1b-MultiSocial,F,0.9592,0.7810,0.8082
5,BLOOMZ-3b-MultiSocial,F,0.9582,0.7843,0.8107
6,XLM-RoBERTa-large-MultiSocial,F,0.9553,0.7840,0.8075
7,mDeBERTa-v3-base-MultiSocial,F,0.9544,0.7652,0.7947
8,BLOOMZ-3b-mixed-Detector,P,0.7553,0.3024,0.5882
9,DetectLLM-LRR,S,0.7464,0.2523,0.6580
10,LLM-Deviation,S,0.7454,0.2497,0.6570


In [22]:

#social media test data - th calibration for macroF1 based on train data
auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    train = data[(data.split == 'train') & (data.domain == 'social_media') & (~data.multi_label.str.contains('gemini'))]
    temp = data[(data.split == 'test') & (data.domain == 'social_media')]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}

  train_labels = [label2id[x] for x in train['label']]
  fpr, tpr, thresholds = roc_curve(train_labels, train['prediction_probs'])
  predictions = [1 if ((x >= thresholds[fpr <= 0.05][-1])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_5'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': rename_detector(model), 'Category': to_category[model], 'AUC ROC': v['auc'], 'MacroF1@5%FPR': v['cr_5']['macro avg']['f1-score']}, index=[0])
  results_all = pd.concat([results_all, temp])
results_all = results_all.sort_values(by=['AUC ROC'], ascending=False).reset_index(drop=True)
results_all.index = results_all.index + 1
results_all.style.format(na_rep=0, precision=4)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 723.77it/s]


,Detector,Category,AUC ROC,MacroF1@5%FPR
1,Llama-3-8b-MultiSocial,F,0.9769,0.8696
2,Mistral-7b-v0.1-MultiSocial,F,0.9768,0.8692
3,Aya-101-MultiSocial,F,0.9731,0.8462
4,Falcon-rw-1b-MultiSocial,F,0.9592,0.7810
5,BLOOMZ-3b-MultiSocial,F,0.9582,0.7843
6,XLM-RoBERTa-large-MultiSocial,F,0.9553,0.7840
7,mDeBERTa-v3-base-MultiSocial,F,0.9544,0.7652
8,BLOOMZ-3b-mixed-Detector,P,0.7553,0.3024
9,DetectLLM-LRR,S,0.7464,0.2523
10,LLM-Deviation,S,0.7454,0.2497


In [23]:
results_all.style.apply(highlight_categories, axis=1).hide('Category', axis=1)

,Detector,AUC ROC,MacroF1@5%FPR
1,Llama-3-8b-MultiSocial,0.976945,0.869627
2,Mistral-7b-v0.1-MultiSocial,0.976791,0.869152
3,Aya-101-MultiSocial,0.973101,0.846166
4,Falcon-rw-1b-MultiSocial,0.959221,0.780969
5,BLOOMZ-3b-MultiSocial,0.958181,0.784282
6,XLM-RoBERTa-large-MultiSocial,0.955259,0.784015
7,mDeBERTa-v3-base-MultiSocial,0.954366,0.765163
8,BLOOMZ-3b-mixed-Detector,0.755251,0.302403
9,DetectLLM-LRR,0.746401,0.252282
10,LLM-Deviation,0.745394,0.249699


In [24]:
print(results_all.reset_index().style.apply(highlight_categories, axis=1).hide('Category', axis=1).format(na_rep=0, precision=4).applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).hide(
    axis=0).to_latex(convert_css=True))

\begin{tabular}{rlrr}
\bfseries index & \bfseries Detector & \bfseries AUC ROC & MacroF1@5%FPR \\
{\cellcolor[HTML]{B6D7A8}} 1 & {\cellcolor[HTML]{B6D7A8}} Llama-3-8b-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9769 & {\cellcolor[HTML]{B6D7A8}} 0.8696 \\
{\cellcolor[HTML]{B6D7A8}} 2 & {\cellcolor[HTML]{B6D7A8}} Mistral-7b-v0.1-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9768 & {\cellcolor[HTML]{B6D7A8}} 0.8692 \\
{\cellcolor[HTML]{B6D7A8}} 3 & {\cellcolor[HTML]{B6D7A8}} Aya-101-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9731 & {\cellcolor[HTML]{B6D7A8}} 0.8462 \\
{\cellcolor[HTML]{B6D7A8}} 4 & {\cellcolor[HTML]{B6D7A8}} Falcon-rw-1b-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9592 & {\cellcolor[HTML]{B6D7A8}} 0.7810 \\
{\cellcolor[HTML]{B6D7A8}} 5 & {\cellcolor[HTML]{B6D7A8}} BLOOMZ-3b-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9582 & {\cellcolor[HTML]{B6D7A8}} 0.7843 \\
{\cellcolor[HTML]{B6D7A8}} 6 & {\cellcolor[HTML]{B6D7A8}} XLM-RoBERTa-large-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.

/tmp/ipykernel_634/666694371.py:1: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(results_all.reset_index().style.apply(highlight_categories, axis=1).hide('Category', axis=1).format(na_rep=0, precision=4).applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).hide(


In [25]:
#social media test data - per LLM

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
 for model,data in detector.items():
  temp_data = data[(data.split == 'test') & (data.domain == 'social_media')]
 if (to_category[model] == 'F') and ('social-media' not in model): continue
 for llm in tqdm(temp_data.multi_label.unique(), total=len(temp_data.multi_label.unique())):
  if llm == 'human': continue
  temp = temp_data[(temp_data.multi_label.str.contains(f'human|{llm}'))]
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[f"{model} {llm}"] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[f"{model} {llm}"][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model.split(' ')[0], 'Generator': model.split(' ')[1], 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model.split(' ')[0], 'Generator': model.split(' ')[1], 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Generator', 'Language']).unstack().style.format(precision=2).highlight_max(props='font-weight: bold;', axis=0).background_gradient(vmin=0.5, axis=None)
results_all['Category'] = results_all['Detector'].map(to_category)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_all['Generator'] = results_all['Generator'].map(rename_generators)
results_perllm = results_all.copy()

  0%|                                                                                           | 0/119 [00:00<?, ?it/s]/tmp/ipykernel_634/714514677.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_all = pd.concat([results_all, temp])
100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 121.54it/s]


In [26]:
#statistical
temp = results_all[(results_all.Category == 'S')].drop(columns=['Category']).set_index(['Detector', 'Generator', 'Language']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
#styles = [dict(selector="th.row_heading.level0", props=[("writing-mode", "vertical-rl"), ('transform', 'rotateZ(180deg)')])]
display(temp)#.set_table_styles(styles)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{').replace('} &', '}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

/tmp/ipykernel_634/836429760.py:5: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{').replace('} &', '}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Generator &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{7}{*}{\rotatebox{90}{\bfseries Binoculars}} & \bfseries Aya-101 & {\cellcolor[HTML]{DBDAEB}} \color[HTML]{000000} 0.70 & {\cellcolor[HTML]{EEE9F3}} \color[HTML]{000000} 0.61 & {\cellcolor[HTML]{F0EAF4}} \color[HTML]{000000} 0.60 & {\cellcolor[HTML]{DEDCEC}} \color[HTML]{000000} 0.69 & {\cellcolor[HTML]{D2D2E7}} \color[HTML]{000000} 0.75 & {\cellcolor[HTML]{C9CEE4}} \color[HTML]{000000} 0.77 & {\cellcolor[HTML]{C1CAE2}} \color[HTM

In [27]:
#pretrained
temp = results_all[(results_all.Category == 'P')].drop(columns=['Category']).set_index(['Detector', 'Generator', 'Language']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{3cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Generator &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{7}{*}{\rotatebox{90}{\parbox{3cm}{\bfseries BLOOMZ-3b-mixed-Detector}}} & \bfseries Aya-101 & {\cellcolor[HTML]{B4C4DF}} \color[HTML]{000000} 0.83 & {\cellcolor[HTML]{A5BDDB}} \color[HTML]{000000} 0.88 & {\cellcolor[HTML]{B9C6E0}} \color[HTML]{000000} 0.82 & {\cellcolor[HTML]{A5BDDB}} \color[HTML]{000000} 0.88 & {\cellcolor[HTML]{AFC1DD}} \color[HTML]{000000} 0.85 & {\cellcolor[HTML]{B9C6E0}} \color[HTML]{000000} 0.82 & {\cellcol

/tmp/ipykernel_634/563979692.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{3cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


In [28]:
#finetuned
temp = results_all[(results_all.Category == 'F')].drop(columns=['Category']).set_index(['Detector', 'Generator', 'Language']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{3cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Generator &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{7}{*}{\rotatebox{90}{\parbox{3cm}{\bfseries Aya-101-MultiSocial}}} & \bfseries Aya-101 & {\cellcolor[HTML]{93B5D6}} \color[HTML]{000000} 0.92 & {\cellcolor[HTML]{7BACD1}} \color[HTML]{000000} 0.98 & {\cellcolor[HTML]{81AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{7BACD1}} \color[HTML]{000000} 0.98 & {\cellcolor[HTML]{81AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{86B0D3}} \color[HTML]{000000} 0.96 & {\cellcolor[HT

/tmp/ipykernel_634/2703689501.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{3cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


In [29]:
#social media test data - per platform

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
 for model,data in detector.items():
  temp_data = data[(data.split == 'test') & (data.domain == 'social_media')]
 if (to_category[model] == 'F') and ('social-media' not in model): continue
 temp_data['source'] = temp_data['source'].str.replace('multisocial_', '')
 for llm in tqdm(temp_data.source.unique(), total=len(temp_data.multi_label.unique())):
  temp = temp_data[(temp_data.source.str.contains(f'{llm}'))]
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[f"{model} {llm}"] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[f"{model} {llm}"][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model.split(' ')[0], 'Platform': model.split(' ')[1], 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model.split(' ')[0], 'Platform': model.split(' ')[1], 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Platform', 'Language']).unstack().style.format(precision=2).highlight_max(props='font-weight: bold;', axis=0).background_gradient(vmin=0.5, axis=None)
results_all['Category'] = results_all['Detector'].map(to_category)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_all['Platform'] = results_all['Platform'].map(rename_platforms)
results_perplatform = results_all.copy()

  0%|                                                                                            | 0/83 [00:00<?, ?it/s]/tmp/ipykernel_634/1587289848.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['source'] = temp_data['source'].str.replace('multisocial_', '')

  1%|█                                                                                   | 1/83 [00:00<01:01,  1.34it/s]/tmp/ipykernel_634/1587289848.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['source'] = temp_data['

In [30]:
#statistical
temp = results_all[(results_all.Category == 'S')].drop(columns=['Category']).set_index(['Detector', 'Platform', 'Language']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Platform &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{5}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries Binoculars}}} & \bfseries Discord & N/A & N/A & {\cellcolor[HTML]{C8CDE4}} \color[HTML]{000000} 0.78 & {\cellcolor[HTML]{CDD0E5}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{C0C9E2}} \color[HTML]{000000} 0.80 & N/A & {\cellcolor[HTML]{ABBFDC}} \color[HTML]{000000} 0.86 & {\cellcolor[HTML]{BCC7E1}} \color[HTML]{000000} 0.81 & {\cellcolor[HTML]{D0D1E6}} \color[HTML]{000000} 0.75 & {\cell

/tmp/ipykernel_634/3243240931.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


In [31]:
#pretrained
temp = results_all[(results_all.Category == 'P')].drop(columns=['Category']).set_index(['Detector', 'Platform', 'Language']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

/tmp/ipykernel_634/323448539.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Platform &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{5}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries BLOOMZ-3b-mixed-Detector}}} & \bfseries Discord & N/A & N/A & {\cellcolor[HTML]{84B0D3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{9FBAD9}} \color[HTML]{000000} 0.89 & {\cellcolor[HTML]{A9BFDC}} \color[HTML]{000000} 0.87 & N/A & {\cellcolor[HTML]{A7BDDB}} \color[HTML]{000000} 0.87 & {\cellcolor[HTML]{9CB9D9}} \color[HTML]{000000} 0.90 & {\cellcolor[HTML]{A9BFDC}} \color[HTML]{000000}

In [32]:
#finetuned
temp = results_all[(results_all.Category == 'F')].drop(columns=['Category']).set_index(['Detector', 'Platform', 'Language']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

/tmp/ipykernel_634/381997529.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Platform &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{5}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries Aya-101-MultiSocial}}} & \bfseries Discord & N/A & N/A & {\cellcolor[HTML]{76AAD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{75A9CF}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{80AED2}} \color[HTML]{000000} 0.97 & N/A & {\cellcolor[HTML]{79ABD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{78ABD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{76AAD0}} \color[HTML]{000000} 1.00

In [33]:

#social media test data - per platform per llm
auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
 for model,data in detector.items():
  temp_data = data[(data.split == 'test') & (data.domain == 'social_media')]
 if (to_category[model] == 'F') and ('social-media' not in model): continue
 temp_data['source'] = temp_data['source'].str.replace('multisocial_', '')
 for llm in tqdm(temp_data.multi_label.unique(), total=len(temp_data.multi_label.unique())):
  if llm == 'human': continue
  temp = temp_data[(temp_data.multi_label.str.contains(f'{llm}') | temp_data.multi_label.str.contains(f'human'))]
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[f"{model} {llm}"] = {'auc': auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.source.unique():
    temp2 = temp[temp.source.str.contains(test_language)].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[f"{model} {llm}"][test_language] = {'auc': auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model.split(' ')[0], 'Generator': model.split(' ')[1], 'Platform': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model.split(' ')[0], 'Generator': model.split(' ')[1], 'Platform': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Platform', 'Language']).unstack().style.format(precision=2).highlight_max(props='font-weight: bold;', axis=0).background_gradient(vmin=0.5, axis=None)
results_all['Category'] = results_all['Detector'].map(to_category)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_all['Platform'] = results_all['Platform'].map(rename_platforms)
results_perplatformperllm = results_all.copy()

  0%|                                                                                            | 0/83 [00:00<?, ?it/s]/tmp/ipykernel_634/1194295248.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['source'] = temp_data['source'].str.replace('multisocial_', '')

  1%|█                                                                                   | 1/83 [00:01<01:35,  1.17s/it]/tmp/ipykernel_634/1194295248.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['source'] = temp_data['

In [34]:
#statistical
temp = results_all[(results_all.Category == 'S')].drop(columns=['Category']).set_index(['Detector', 'Generator', 'Platform']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

\begin{tabular}{llrrrrrr}
 &  & \multicolumn{6}{r}{\bfseries AUC ROC} \\
 & Platform & \bfseries Discord & \bfseries Gab & \bfseries Telegram & \bfseries Twitter & \bfseries WhatsApp & \bfseries {all} \\
Detector & Generator &  &  &  &  &  &  \\
\hline
\multirow[c]{7}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries Binoculars}}} & \bfseries Mistral-7B-Instruct-v0.2 & {\cellcolor[HTML]{C9CEE4}} \color[HTML]{000000} 0.77 & {\cellcolor[HTML]{EDE8F3}} \color[HTML]{000000} 0.62 & {\cellcolor[HTML]{DFDDEC}} \color[HTML]{000000} 0.68 & {\cellcolor[HTML]{DFDDEC}} \color[HTML]{000000} 0.68 & {\cellcolor[HTML]{E3E0EE}} \color[HTML]{000000} 0.67 & {\cellcolor[HTML]{E1DFED}} \color[HTML]{000000} 0.68 \\
\bfseries  & \bfseries aya-101 & {\cellcolor[HTML]{CED0E6}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{E7E3F0}} \color[HTML]{000000} 0.65 & {\cellcolor[HTML]{E0DDED}} \color[HTML]{000000} 0.68 & {\cellcolor[HTML]{DCDAEB}} \color[HTML]{000000} 0.70 & {\cellcolor[HTML]{D8D7E9}} \color[HTML]{000000} 0.72 

/tmp/ipykernel_634/660990578.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


In [35]:
#pretrained
temp = results_all[(results_all.Category == 'P')].drop(columns=['Category']).set_index(['Detector', 'Generator', 'Platform']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

/tmp/ipykernel_634/2287834888.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


\begin{tabular}{llrrrrrr}
 &  & \multicolumn{6}{r}{\bfseries AUC ROC} \\
 & Platform & \bfseries Discord & \bfseries Gab & \bfseries Telegram & \bfseries Twitter & \bfseries WhatsApp & \bfseries {all} \\
Detector & Generator &  &  &  &  &  &  \\
\hline
\multirow[c]{7}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries BLOOMZ-3b-mixed-Detector}}} & \bfseries Mistral-7B-Instruct-v0.2 & {\cellcolor[HTML]{A8BEDC}} \color[HTML]{000000} 0.87 & {\cellcolor[HTML]{EDE7F2}} \color[HTML]{000000} 0.62 & {\cellcolor[HTML]{CED0E6}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{DBDAEB}} \color[HTML]{000000} 0.70 & {\cellcolor[HTML]{C5CCE3}} \color[HTML]{000000} 0.78 & {\cellcolor[HTML]{D3D4E7}} \color[HTML]{000000} 0.74 \\
\bfseries  & \bfseries aya-101 & {\cellcolor[HTML]{93B5D6}} \color[HTML]{000000} 0.92 & {\cellcolor[HTML]{CED0E6}} \color[HTML]{000000} 0.75 & {\cellcolor[HTML]{ADC1DD}} \color[HTML]{000000} 0.85 & {\cellcolor[HTML]{BBC7E0}} \color[HTML]{000000} 0.81 & {\cellcolor[HTML]{B0C2DE}} \color[HTML]

In [36]:
#finetuned
temp = results_all[(results_all.Category == 'F')].drop(columns=['Category']).set_index(['Detector', 'Generator', 'Platform']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

\begin{tabular}{llrrrrrr}
 &  & \multicolumn{6}{r}{\bfseries AUC ROC} \\
 & Platform & \bfseries Discord & \bfseries Gab & \bfseries Telegram & \bfseries Twitter & \bfseries WhatsApp & \bfseries {all} \\
Detector & Generator &  &  &  &  &  &  \\
\hline
\multirow[c]{7}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries Aya-101-MultiSocial}}} & \bfseries Mistral-7B-Instruct-v0.2 & {\cellcolor[HTML]{75A9CF}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{7EADD1}} \color[HTML]{000000} 0.98 & {\cellcolor[HTML]{76AAD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{76AAD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{75A9CF}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{78ABD0}} \color[HTML]{000000} 0.99 \\
\bfseries  & \bfseries aya-101 & {\cellcolor[HTML]{79ABD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{99B8D8}} \color[HTML]{000000} 0.91 & {\cellcolor[HTML]{81AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{83AFD3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{88B1D4}} \color[HTML]{0000

/tmp/ipykernel_634/2577362353.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{7}{*}{', '\\multirow[c]{7}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


## RQ1: Zero-shot MGTD

In [37]:
#due to low performance
to_exclude = ['ruRoBERTa-ruatd-binary', 'RoBERTa-large-OpenAI-Detector',
       'Longformer Detector']

In [38]:
temp_results_all = results_all.copy()
results_all = results_perplatform.copy()

In [39]:
results_all = results_all[~results_all.Detector.isin(to_exclude)]

In [40]:
#per platform mean per detector category
temp = pd.DataFrame(multidomain[multidomain.domain=='social_media'][['source', 'language']].value_counts())
temp = temp.reset_index()
temp['source'] = temp['source'].str.replace('multisocial_', '')
temp = temp.groupby(['source', 'language']).sum().unstack().fillna(0).astype(int) > 2000 #if there are enough samples (~ 250 of each generator)
enough_platform_language_samples = pd.concat([temp, temp])
enough_platform_language_samples[('count', '{all}')] = True

temp = results_all[(results_all.Category != 'F')].groupby(['Category', 'Platform', 'Language']).mean(numeric_only=True).unstack()
enough_platform_language_samples.index = temp.index
enough_platform_language_samples.columns = temp.columns
temp[~enough_platform_language_samples] = 'nan'
platforms = temp.copy()
#temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None, gmap=temp).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
#display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


/tmp/ipykernel_634/3129301977.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp[~enough_platform_language_samples] = 'nan'


In [41]:
#social media test data

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    temp = data[(data.split == 'test') & (data.domain == 'social_media')]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[model][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model, 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model, 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).style.format(precision=2).apply(lambda x: ["background-color: lightyellow" if v >= 0.6 else "" for v in x], axis = 0, subset='AUC ROC').highlight_max(props='font-weight: bold;', axis=0)
results_all['Category'] = results_all['Detector'].map(to_category)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_perlanguage = results_all.copy()

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 120.02it/s]


In [42]:
temp = results_all[results_all.Category != 'F'].set_index(['Category', 'Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).sort_index().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('nan', 'N/A'))

/tmp/ipykernel_634/2459050323.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = results_all[results_all.Category != 'F'].set_index(['Category', 'Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).sort_index().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')


/tmp/ipykernel_634/2459050323.py:3: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('nan', 'N/A'))


\begin{tabular}{llrrrrrrrrrrrrrrrrrrrrrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Category & Detector &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{5}{*}{\bfseries P} & \bfseries BLOOMZ-3b-mixed-Detector & {\cellcolor[HTML]{C4CBE3}} \color[HTML]{000000} 0.79 & {\cellcolor[HTML]{D2D3E7}} \color[HTML]{000000} 0.74 & {\cellcolor[HTML]{C2CBE2}} \color[HTML]{000000} 0.79 & {\cellcolor[HTML]{C1CAE2}} \color[HTML]{000000} 0.80 & {\cellcolor[HTML]{CACEE5}} \color[HTML]{000000} 0.77 & {\cellcolor[HTML]{CCCFE5}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{BFC9E1}} \color[HTML]{000000

In [43]:
results_all = results_all[~results_all.Detector.isin(to_exclude)]

In [44]:
#mean per detector category
temp = pd.DataFrame(multidomain[multidomain.domain=='social_media'][['language']].value_counts())
temp = temp.reset_index()
temp = temp.groupby(['language']).sum().unstack().fillna(0).astype(int) > 2000 #if there are enough samples (~ 250 of each generator)
enough_platform_language_samples = pd.concat([temp, temp], axis=1)
enough_platform_language_samples = enough_platform_language_samples.T
enough_platform_language_samples['{all}'] = True

temp = results_all[(results_all.Category != 'F')].groupby(['Category', 'Language']).mean(numeric_only=True).unstack()
enough_platform_language_samples.index = temp.index
enough_platform_language_samples.columns = temp.columns
temp[~enough_platform_language_samples] = 'nan'
#temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None, gmap=temp).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
#display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{5}{*}{', '\\multirow[c]{5}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))
temp['Platform'] = '{all}'


/tmp/ipykernel_634/1941271779.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp[~enough_platform_language_samples] = 'nan'


In [45]:
temp = pd.concat([platforms.reset_index(), temp.reset_index()]).sort_values(['Category', 'Platform']).set_index(['Category', 'Platform'])
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None, gmap=temp).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('nan', 'N/A'))

/tmp/ipykernel_634/2992061236.py:2: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None, gmap=temp).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')


/tmp/ipykernel_634/2992061236.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('nan', 'N/A'))


\begin{tabular}{llllllllrrlllllllrllllllr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Category & Platform &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{6}{*}{\bfseries P} & \bfseries Discord & {\cellcolor[HTML]{000000}} \color[HTML]{000000} {\cellcolor{white}} N/A & {\cellcolor[HTML]{000000}} \color[HTML]{000000} {\cellcolor{white}} N/A & {\cellcolor[HTML]{94B6D7}} \color[HTML]{000000} 0.92 & {\cellcolor[HTML]{BCC7E1}} \color[HTML]{000000} 0.81 & {\cellcolor[HTML]{ABBFDC}} \color[HTML]{000000} 0.86 & {\cellcolor[HTML]{000000}} \color[HTML]{000000} {\cellcolor{white}} N/A & {\cell

In [46]:
results_all[(results_all.Category == 'S')].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack().corr().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')

/tmp/ipykernel_634/3261554257.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  results_all[(results_all.Category == 'S')].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack().corr().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')


### Statistics

In [47]:
#statistical significance of per-language per-platform results between statistical and pretrained detectors
temp = results_perplatform.copy()
temp = temp[temp['Category'] != 'F'].reset_index(drop=True)
temp = temp[~temp.Detector.isin(to_exclude)]
languages = temp.Language.unique()
temp = temp.pivot(index=['Category', 'Platform', 'Detector'], columns='Language', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])

res_df = pd.DataFrame()
for (src, trg) in itertools.combinations_with_replacement(temp.Category.unique(), 2):
 if src != trg: continue
 for (psrc, ptrg) in itertools.combinations_with_replacement(temp.Platform.unique(), 2):
  if psrc == ptrg: continue
  for lang in languages[languages == '{all}']:
   try:
    print(f"\nTest language: {lang}, ({ptrg}, {psrc}), ({trg}, {src})")
    res = pg.ttest(temp[(temp["Category"] == trg) & (temp["Platform"] == ptrg)][lang], temp[(temp["Category"] == src) & (temp["Platform"] == psrc)][lang], paired=True)
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass


Test language: {all}, (Gab, Discord), (P, P)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.877336,1,two-sided,0.212940,[-0.85 0.54],3.133956,1.271,0.271963



Test language: {all}, (Telegram, Discord), (P, P)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.755065,1,two-sided,0.109525,[-0.26 0.1 ],1.296543,1.772,0.116036



Test language: {all}, (Twitter, Discord), (P, P)


...

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,6.616423,4,two-sided,0.002706,[0. 0.01],1.030571,18.195,0.420866



Test language: {all}, (WhatsApp, Telegram), (S, S)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.087172,4,two-sided,0.338086,[-0.06 0.02],0.704080,0.611,0.229811



Test language: {all}, (WhatsApp, Twitter), (S, S)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.491922,4,two-sided,0.209995,[-0.06 0.02],0.938104,0.824,0.362619


In [48]:
#statistical significance of per-language results between statistical and pretrained detectors
temp = results_perlanguage.copy()
temp = temp[temp['Category'] != 'F'].reset_index(drop=True)
temp = temp[~temp.Detector.isin(to_exclude)]
languages = temp.Language.unique()
temp = temp.pivot(index=['Category', 'Detector'], columns='Language', values='AUC ROC').reset_index()

res_df = pd.DataFrame()
for (src, trg) in itertools.combinations_with_replacement(temp.Category.unique(), 2):
  for lang in languages:
   if src == trg: continue
   print(f"\nTest language: {lang}, ({trg}, {src})")
   res = pg.ttest(temp[temp["Category"] == trg][lang], temp[temp["Category"] == src][lang], paired=True)
   display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))


Test language: {all}, (S, P)


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.178734,1.090455,two-sided,0.885510,[-0.2 0.2],0.240264,0.574,0.056449



Test language: ro, (S, P)


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


...

/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,6.862711,4.954537,two-sided,0.001041,[0.08 0.17],3.712047,22.583,0.938458



Test language: zh, (S, P)


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.264622,1.067859,two-sided,0.833141,[-0.63 0.66],0.368117,0.58,0.065193


In [49]:
#statistical significance of per-language results between EN and non-EN languages
temp = results_perlanguage.copy()
temp = temp[temp['Category'] != 'F'].reset_index(drop=True)
#temp = temp[~temp.Detector.isin(to_exclude)]
languages = temp.Language.unique()
display(temp.head())
#temp = temp.pivot(index=['Category', 'Detector'], columns='Language', values='AUC ROC').reset_index()
#display(temp.head())

res_df = pd.DataFrame()
for lang in languages:
   if lang == 'en': continue
   print(f"\nTest language: en vs {lang}")
   res = pg.ttest(temp[temp['Language'] == 'en']['AUC ROC'], temp[temp['Language'] == lang]['AUC ROC'], paired=True)
   display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

print(f"\nTest language: en vs others")
res = pg.ttest(temp[temp['Language'] == 'en']['AUC ROC'], temp[(temp['Language'] != 'en') & (temp['Language'] != '{all}')]['AUC ROC'], paired=True)
display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

print('*' * 10, 'pretrained')
res_df = pd.DataFrame()
for lang in languages:
   if lang == 'en': continue
   print(f"\nTest language: en vs {lang}")
   res = pg.ttest(temp[(temp.Category == 'P') & (temp['Language'] == 'en')]['AUC ROC'], temp[(temp.Category == 'P') & (temp['Language'] == lang)]['AUC ROC'], paired=True)
   display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

print(f"\nTest language: en vs others")
res = pg.ttest(temp[(temp.Category == 'P') & (temp['Language'] == 'en')]['AUC ROC'], temp[(temp.Category == 'P') & (temp['Language'] != 'en') & (temp['Language'] != '{all}')]['AUC ROC'], paired=True)
display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

print('*' * 10, 'statistical')
res_df = pd.DataFrame()
for lang in languages:
   if lang == 'en': continue
   print(f"\nTest language: en vs {lang}")
   res = pg.ttest(temp[(temp.Category == 'S') & (temp['Language'] == 'en')]['AUC ROC'], temp[(temp.Category == 'S') & (temp['Language'] == lang)]['AUC ROC'], paired=True)
   display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

print(f"\nTest language: en vs others")
res = pg.ttest(temp[(temp.Category == 'S') & (temp['Language'] == 'en')]['AUC ROC'], temp[(temp.Category == 'S') & (temp['Language'] != 'en') & (temp['Language'] != '{all}')]['AUC ROC'], paired=True)
display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

,Detector,Language,AUC ROC,Category
0,BLOOMZ-3b-mixed-Detector,{all},0.755251,P
1,BLOOMZ-3b-mixed-Detector,ro,0.662115,P
2,BLOOMZ-3b-mixed-Detector,cs,0.796334,P
3,BLOOMZ-3b-mixed-Detector,pl,0.782974,P
4,BLOOMZ-3b-mixed-Detector,hr,0.773144,P



Test language: en vs {all}


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.923321,9,two-sided,0.000821,[0.05 0.14],0.669497,48.181,0.472387



Test language: en vs ro


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.913002,9,two-sided,0.088037,[-0.02 0.18],0.444895,1.158,0.242505


...


Test language: en vs zh


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.253167,4,two-sided,0.087338,[-0.01 0.07],1.667215,1.496,0.793465



Test language: en vs others


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.760675,31.236026,two-sided,0.452555,[-0.01 0.02],0.092591,0.491,0.054617


In [50]:
#based on mean AUC across detectors calculated on EN and nonEN samples
non_en = [0.7442879083629403, 0.744780307053057, 0.7420986995803567, 0.7368291096667234, 0.7171516831066395, 0.7503867288357737, 0.7074464884477927]
en = [0.7961044832461726, 0.8008579759826806, 0.7748092906493541, 0.8014880529805979, 0.8015463632661355, 0.8034267459856592, 0.8970331400804356]

print(f"\nTest language: en vs others")
res = pg.ttest(en, non_en, paired=True)
display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))


Test language: en vs others


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.838321,6,two-sided,0.008576,[0.03 0.12],2.532048,7.562,0.999748


## RQ2: Fine-tuned MGTD

In [51]:
#social media test data

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    temp = data[(data.split == 'test') & (data.domain == 'social_media')]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[model][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model, 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model, 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).style.format(precision=2).apply(lambda x: ["background-color: lightyellow" if v >= 0.6 else "" for v in x], axis = 0, subset='AUC ROC').highlight_max(props='font-weight: bold;', axis=0)
results_all['Category'] = results_all['Detector'].map(to_category)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_perlanguage = results_all.copy()

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 126.64it/s]


In [52]:
results_all = results_perlanguage
temp = results_all[results_all.Category == 'F'].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).sort_index().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('nan', 'N/A'))

/tmp/ipykernel_634/1163991209.py:2: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = results_all[results_all.Category == 'F'].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).sort_index().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')


\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrrr}
 & \multicolumn{23}{r}{\bfseries AUC ROC} \\
Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\bfseries Aya-101-MultiSocial & {\cellcolor[HTML]{81AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{78ABD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{7EADD1}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{7BACD1}} \color[HTML]{000000} 0.98 & {\cellcolor[HTML]{80AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{80AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{7EADD1}} \color[HTML]{000000} 0.98 & {\cellcolor[HTML]{7DACD1}} \color[HTML]{000000} 0.98 & 

/tmp/ipykernel_634/1163991209.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('nan', 'N/A'))


In [53]:
results_all.Language.unique()

array(['{all}', 'ro', 'cs', 'pl', 'hr', 'ar', 'en', 'pt', 'de', 'es',
       'nl', 'ca', 'uk', 'el', 'sk', 'hu', 'ga', 'et', 'gd', 'bg', 'ru',
       'sl', 'zh'], dtype=object)

In [54]:
sorted_languages = [
                    'el',
                    'ar', 'zh',
                    'ca', 'es', 'pt',
                    'en', 'ga', 'gd',
                    'hu', 'et',
                    'de', 'nl',
                    'cs', 'pl', 'sk',
                    'hr', 'sl',
                    'ro',
                    'bg', 'uk',  'ru',
                    ]
temp = results_all[results_all.Category == 'F'].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack()[[('AUC ROC', x) for x in sorted_languages]].corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))

/tmp/ipykernel_634/1636405447.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  temp = results_all[results_all.Category == 'F'].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack()[[('AUC ROC', x) for x in sorted_languages]].corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
/tmp/ipykernel_634/1636405447.py:13: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = results_all[results_all.Category == 'F'].drop(columns=['Category']).set_index(['Detector', 'Language']).unstack()[[('AUC ROC', x) for x in sorted_languages]].corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cma

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrr}
 & \multicolumn{22}{r}{\bfseries AUC ROC} \\
Language & \bfseries el & \bfseries ar & \bfseries zh & \bfseries ca & \bfseries es & \bfseries pt & \bfseries en & \bfseries ga & \bfseries gd & \bfseries hu & \bfseries et & \bfseries de & \bfseries nl & \bfseries cs & \bfseries pl & \bfseries sk & \bfseries hr & \bfseries sl & \bfseries ro & \bfseries bg & \bfseries uk & \bfseries ru \\
Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\bfseries el & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{7CC87C}} \color[HTML]{000000} 0.92 & {\cellcolor[HTML]{81CA81}} \color[HTML]{000000} 0.84 & {\cellcolor[HTML]{7DC87E}} \color[HTML]{000000} 0.90 & {\cellcolor[HTML]{7FC97F}} \color[HTML]{000000} 0.88 & {\cellcolor[HTML]{86CC85}} \color[HTML]{000000} 0.81 & {\cellcolor[HTML]{81CA81}} \color[HTML]{000000} 0.85 & {\cellcolor[HTML]{83CB82}} \color[HTML]{000000} 0.83 & {\cellcolor[HTML]{7AC77B}} \color[HTML]

/tmp/ipykernel_634/1636405447.py:15: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))


In [55]:
#social media telegram test data - per language of telegram trained mono and multilingually traned EN,ES,RU detectors

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    temp = data[(data.split == 'test') & (data.domain == 'social_media')]
  if ('cross-language' not in model): continue
  temp['source'] = temp['source'].str.replace('multisocial_', '')
  temp = temp[temp.source == 'telegram']
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  temp['prediction_probs']= temp['prediction_probs'].astype(float)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'].astype(float))
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[model][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': extract_language(model).split('_')[0], 'Train Language': extract_language(model).split('_')[1], 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': extract_language(model).split('_')[0], 'Train Language': extract_language(model).split('_')[1], 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).style.format(precision=2).apply(lambda x: ["background-color: lightyellow" if v >= 0.6 else "" for v in x], axis = 0, subset='AUC ROC').highlight_max(props='font-weight: bold;', axis=0)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_language = results_all.copy()

 12%|██████████                                                                         | 10/83 [00:00<00:06, 11.77it/s]/tmp/ipykernel_634/1074113524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['source'] = temp['source'].str.replace('multisocial_', '')
/tmp/ipykernel_634/1074113524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['source'] = temp['source'].str.replace('multisocial_', '')
 14%|████████████                                                                       | 12/83 [00:

In [56]:
results_all = results_language

In [57]:
temp = pd.DataFrame(multidomain[(multidomain.domain=='social_media') & multidomain.source.str.contains('telegram')][['language']].value_counts())
temp = temp.reset_index()
temp = temp.groupby(['language']).sum().unstack().fillna(0).astype(int) > 2000 #if there are enough samples (~ 250 of each generator)
temp.index = temp.index.droplevel(0)
temp['{all}'] = True
enough_platform_language_samples = temp

In [58]:
#temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack()
temp = results_all[~results_all['Train Language'].str.contains('x3')].set_index(['Detector', 'Train Language', 'Language']).unstack()
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.loc[[(x,y) for x,y in temp.index if ('llama' in x.lower()) or ('mistral' in x.lower()) or ('mdeberta' in x.lower())]]
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))

\begin{tabular}{llrrrrrrrrrllrrrrrrrllrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Train Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{4}{*}{\rotatebox{90}{\parbox{1.8cm}{\bfseries Llama-3-8b}}} & \bfseries en & {\cellcolor[HTML]{ADC1DD}} \color[HTML]{000000} 0.85 & {\cellcolor[HTML]{83AFD3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{CCCFE5}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{9FBAD9}} \color[HTML]{000000} 0.89 & {\cellcolor[HTML]{9FBAD9}} \color[HTML]{000000} 0.89 & {\cellcolor[HTML]{9AB8D8}} \color[HTML]{000000} 0.90 & {\cellcolor[HTML]{80A

/tmp/ipykernel_634/2521415468.py:7: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))


In [59]:
#temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack()
temp = results_all[~results_all['Train Language'].str.contains('x3')].set_index(['Detector', 'Train Language', 'Language']).unstack()
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))

\begin{tabular}{llrrrrrrrrrllrrrrrrrllrrr}
 &  & \multicolumn{23}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Detector & Train Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{4}{*}{\rotatebox{90}{\parbox{1.8cm}{\bfseries Aya-101}}} & \bfseries en & {\cellcolor[HTML]{9FBAD9}} \color[HTML]{000000} 0.89 & {\cellcolor[HTML]{7EADD1}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{B1C2DE}} \color[HTML]{000000} 0.84 & {\cellcolor[HTML]{80AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{93B5D6}} \color[HTML]{000000} 0.92 & {\cellcolor[HTML]{88B1D4}} \color[HTML]{000000} 0.95 & {\cellcolor[HTML]{80AED2

/tmp/ipykernel_634/358942449.py:6: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))


In [60]:
temp = results_all[results_all['Train Language'] != '{en-es-ru-x3}']
temp['Train'] = 'monolingual'
temp.loc[temp['Train Language'] == '{en-es-ru}', 'Train'] = 'multilingual'
#temp = temp.set_index(['Train', 'Detector', 'Language']).unstack().groupby('Train').mean().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
temp = temp.groupby(['Train', 'Language'])['AUC ROC'].mean().unstack()
temp[[x for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

/tmp/ipykernel_634/2440888058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Train'] = 'monolingual'


Language,ar,bg,ca,cs,de,el,en,es,et,ga,gd,hr,hu,nl,pl,pt,ro,ru,sk,sl,uk,zh,{all}
Train,,,,,,,,,,,,,,,,,,,,,,,
monolingual,0.81,0.91,0.79,0.89,0.87,0.89,0.91,0.88,0.90,nan,nan,0.92,0.94,0.82,0.88,0.90,0.92,0.90,nan,nan,0.84,0.73,0.86
multilingual,0.89,0.94,0.86,0.93,0.91,0.93,0.95,0.93,0.93,nan,nan,0.94,0.97,0.86,0.91,0.94,0.95,0.93,nan,nan,0.89,0.86,0.91


In [61]:
temp = results_all.set_index(['Train Language', 'Detector', 'Language']).unstack().groupby('Train Language').mean().drop(index=['{en-es-ru-x3}'])
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

In [62]:
autoregressive = [x for x in results_all.Detector.unique() if ('aya' not in x.lower()) and ('mdeberta' not in x.lower()) and ('xlm' not in x.lower())]

In [63]:
#autoregressive decoder-only models
temp = results_all[results_all.Detector.isin(autoregressive)].set_index(['Train Language', 'Detector', 'Language']).unstack().groupby('Train Language').mean().drop(index=['{en-es-ru-x3}'])
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}'))

\begin{tabular}{lrrrrrrrrrllrrrrrrrllrrr}
 & \multicolumn{23}{r}{\bfseries AUC ROC} \\
Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Train Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\bfseries en & {\cellcolor[HTML]{C6CCE3}} \color[HTML]{000000} 0.78 & {\cellcolor[HTML]{ABBFDC}} \color[HTML]{000000} 0.86 & {\cellcolor[HTML]{C9CEE4}} \color[HTML]{000000} 0.77 & {\cellcolor[HTML]{A7BDDB}} \color[HTML]{000000} 0.87 & {\cellcolor[HTML]{A5BDDB}} \color[HTML]{000000} 0.88 & {\cellcolor[HTML]{A4BCDA}} \color[HTML]{000000} 0.88 & {\cellcolor[HTML]{84B0D3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{B0C2DE}} \color[HTML]{000000} 0.85 & {\cellcolor

/tmp/ipykernel_634/2204231633.py:6: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}'))


In [64]:
#correlation between models
temp = results_all[(~results_all['Train Language'].str.contains('{en-es-ru-x3}'))].set_index(['Train Language', 'Language', 'Detector']).unstack().groupby('Train Language').mean()
#temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
#temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
#display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}'))

In [65]:
temp

AUC ROC                                                    \
Detector         Aya-101 BLOOMZ-3b Falcon-rw-1b Llama-3-8b Mistral-7b-v0.1   
Train Language                                                               
en              0.932335  0.843346     0.841710   0.891368        0.826079   
es              0.937753  0.820825     0.846767   0.870279        0.822502   
ru              0.935936  0.754461     0.799556   0.825385        0.827751   
{en-es-ru}      0.939851  0.871641     0.892501   0.940352        0.903409   

                                                   
Detector       XLM-RoBERTa-large mDeBERTa-v3-base  
Train Language                                     
en                      0.897961         0.899458  
es                      0.913116         0.906526  
ru                      0.930225         0.916440  
{en-es-ru}              0.937756         0.913727

In [66]:
sorted_detectors = [
                    'BLOOMZ-3b', 'Falcon-rw-1b', 'Llama-3-8b', 'Mistral-7b-v0.1',
                    'Aya-101', 'XLM-RoBERTa-large', 'mDeBERTa-v3-base'
                    ]
temp[[('AUC ROC', x) for x in sorted_detectors]].corr().style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')

/tmp/ipykernel_634/2497660348.py:5: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp[[('AUC ROC', x) for x in sorted_detectors]].corr().style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')


In [67]:
#not autoregressive models (seq2seq or masked)
temp = results_all[~results_all.Detector.isin(autoregressive)].set_index(['Train Language', 'Detector', 'Language']).unstack().groupby('Train Language').mean().drop(index=['{en-es-ru-x3}'])
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}'))

\begin{tabular}{lrrrrrrrrrllrrrrrrrllrrr}
 & \multicolumn{23}{r}{\bfseries AUC ROC} \\
Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all} \\
Train Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\bfseries en & {\cellcolor[HTML]{AFC1DD}} \color[HTML]{000000} 0.85 & {\cellcolor[HTML]{88B1D4}} \color[HTML]{000000} 0.95 & {\cellcolor[HTML]{C2CBE2}} \color[HTML]{000000} 0.79 & {\cellcolor[HTML]{84B0D3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{A4BCDA}} \color[HTML]{000000} 0.88 & {\cellcolor[HTML]{8EB3D5}} \color[HTML]{000000} 0.93 & {\cellcolor[HTML]{83AFD3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{9AB8D8}} \color[HTML]{000000} 0.90 & {\cellcolor

/tmp/ipykernel_634/2127811620.py:6: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}'))


In [68]:
sorted_languages = [
                    'el',
                    'ar', 'zh',
                    'ca', 'es', 'pt',
                    'en', 'ga', 'gd',
                    'hu', 'et',
                    'de', 'nl',
                    'cs', 'pl', 'sk',
                    'hr', 'sl',
                    'ro',
                    'bg', 'uk',  'ru',
                    ]
temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack()[[('AUC ROC', x) for x in sorted_languages]].corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
#temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack().corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))

/tmp/ipykernel_634/4163803263.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack()[[('AUC ROC', x) for x in sorted_languages]].corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
/tmp/ipykernel_634/4163803263.py:13: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack()[[('AUC ROC', x) for x in sorted_languages]].corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lam

/tmp/ipykernel_634/4163803263.py:16: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))


\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrr}
 & \multicolumn{22}{r}{\bfseries AUC ROC} \\
Language & \bfseries el & \bfseries ar & \bfseries zh & \bfseries ca & \bfseries es & \bfseries pt & \bfseries en & \bfseries ga & \bfseries gd & \bfseries hu & \bfseries et & \bfseries de & \bfseries nl & \bfseries cs & \bfseries pl & \bfseries sk & \bfseries hr & \bfseries sl & \bfseries ro & \bfseries bg & \bfseries uk & \bfseries ru \\
Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\bfseries el & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{8BCF89}} \color[HTML]{000000} 0.75 & {\cellcolor[HTML]{8ACE88}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{9FD899}} \color[HTML]{000000} 0.53 & {\cellcolor[HTML]{9FD899}} \color[HTML]{000000} 0.53 & {\cellcolor[HTML]{90D18D}} \color[HTML]{000000} 0.69 & {\cellcolor[HTML]{8DD08A}} \color[HTML]{000000} 0.72 & {\cellcolor[HTML]{90D18D}} \color[HTML]{000000} 0.70 & {\cellcolor[HTML]{000000}} \color[HTML]

In [69]:

#social media telegram test data - per language-family of telegram trained mono and multilingually traned EN,ES,RU detectors
auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    temp = data[(data.split == 'test') & (data.domain == 'social_media')]
  if ('cross-language' not in model): continue
  temp['source'] = temp['source'].str.replace('multisocial_', '')
  temp = temp[temp.source == 'telegram']
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  temp['language'] = temp['language'].map(to_language_family)
  temp['prediction_probs']= temp['prediction_probs'].astype(float)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'].astype(float))
  auc_dict[model] = {'auc': auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[model][test_language] = {'auc': auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': extract_language(model).split('_')[0], 'Train Language': extract_language(model).split('_')[1], 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': extract_language(model).split('_')[0], 'Train Language': extract_language(model).split('_')[1], 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).style.format(precision=2).apply(lambda x: ["background-color: lightyellow" if v >= 0.6 else "" for v in x], axis = 0, subset='AUC ROC').highlight_max(props='font-weight: bold;', axis=0)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_language_family = results_all.copy()

 12%|██████████                                                                         | 10/83 [00:00<00:06, 11.07it/s]/tmp/ipykernel_634/3236697026.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['source'] = temp['source'].str.replace('multisocial_', '')
/tmp/ipykernel_634/3236697026.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['source'] = temp['source'].str.replace('multisocial_', '')
 14%|████████████                                                                       | 12/83 [00:

In [70]:
results_all = results_language_family

In [71]:
temp = pd.DataFrame(multidomain[(multidomain.domain=='social_media') & multidomain.source.str.contains('telegram')][['language']].value_counts())
temp = temp.reset_index()
temp['language'] = temp['language'].map(to_language_family)
temp = temp.groupby(['language']).sum().unstack().fillna(0).astype(int) > 2000 #if there are enough samples (~ 250 of each generator)
temp.index = temp.index.droplevel(0)
temp['{all}'] = True
enough_platform_language_samples = temp

#temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack()
temp = results_all[~results_all['Train Language'].str.contains('x3')].set_index(['Detector', 'Train Language', 'Language']).unstack()
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))

\begin{tabular}{lllrrrrrrrrr}
 &  & \multicolumn{10}{r}{\bfseries AUC ROC} \\
 & Language & \bfseries Celtic & \bfseries Germanic & \bfseries Greek & \bfseries Romance & \bfseries Semitic & \bfseries Sino-Tibetan & \bfseries Slavic-Cyrillic & \bfseries Slavic-Latin & \bfseries Uralic & \bfseries {all} \\
Detector & Train Language &  &  &  &  &  &  &  &  &  &  \\
\hline
\multirow[c]{4}{*}{\rotatebox{90}{\parbox{1.8cm}{\bfseries Aya-101}}} & \bfseries en & N/A & {\cellcolor[HTML]{93B5D6}} \color[HTML]{000000} 0.92 & {\cellcolor[HTML]{88B1D4}} \color[HTML]{000000} 0.95 & {\cellcolor[HTML]{91B5D6}} \color[HTML]{000000} 0.93 & {\cellcolor[HTML]{9FBAD9}} \color[HTML]{000000} 0.89 & {\cellcolor[HTML]{97B7D7}} \color[HTML]{000000} 0.91 & {\cellcolor[HTML]{8EB3D5}} \color[HTML]{000000} 0.94 & {\cellcolor[HTML]{88B1D4}} \color[HTML]{000000} 0.95 & {\cellcolor[HTML]{7DACD1}} \color[HTML]{000000} 0.98 & {\cellcolor[HTML]{8EB3D5}} \color[HTML]{000000} 0.93 \\
\bfseries  & \bfseries es & N/A & {\cel

/tmp/ipykernel_634/72917441.py:14: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))


In [72]:
temp = results_all.set_index(['Train Language', 'Detector', 'Language']).unstack().groupby('Train Language').mean().drop(index=['{en-es-ru-x3}'])
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

In [73]:
#autoregressive decoder-only models
temp = results_all[results_all.Detector.isin(autoregressive)].set_index(['Train Language', 'Detector', 'Language']).unstack().groupby('Train Language').mean().drop(index=['{en-es-ru-x3}'])
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

In [74]:
#not autoregressive models (seq2seq or masked)
temp = results_all[~results_all.Detector.isin(autoregressive)].set_index(['Train Language', 'Detector', 'Language']).unstack().groupby('Train Language').mean().drop(index=['{en-es-ru-x3}'])
temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

In [75]:
temp = results_all[~results_all['Train Language'].str.contains('x3')]
temp = temp[~temp.Language.isin(enough_platform_language_samples[~enough_platform_language_samples].index.tolist())]
temp = temp.set_index(['Detector', 'Train Language', 'Language']).unstack()
temp = temp.replace('nan', pd.NA).corr().reset_index().drop(columns=['level_0']).set_index('Language')
#temp[[('AUC ROC', x) for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp = temp.style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
#temp = results_all.set_index(['Detector', 'Train Language', 'Language']).unstack().corr(numeric_only=True).reset_index().drop(columns=['level_0']).set_index('Language').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))

/tmp/ipykernel_634/4165235407.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  temp = temp.replace('nan', pd.NA).corr().reset_index().drop(columns=['level_0']).set_index('Language')
/tmp/ipykernel_634/4165235407.py:6: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = temp.style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')


\begin{tabular}{lrrrrrrrrr}
 & \multicolumn{9}{r}{\bfseries AUC ROC} \\
Language & \bfseries Germanic & \bfseries Greek & \bfseries Romance & \bfseries Semitic & \bfseries Sino-Tibetan & \bfseries Slavic-Cyrillic & \bfseries Slavic-Latin & \bfseries Uralic & \bfseries {all} \\
Language &  &  &  &  &  &  &  &  &  \\
\bfseries Germanic & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{8DD08A}} \color[HTML]{000000} 0.73 & {\cellcolor[HTML]{8ACE88}} \color[HTML]{000000} 0.76 & {\cellcolor[HTML]{86CC85}} \color[HTML]{000000} 0.81 & {\cellcolor[HTML]{97D492}} \color[HTML]{000000} 0.62 & {\cellcolor[HTML]{A9DCA3}} \color[HTML]{000000} 0.39 & {\cellcolor[HTML]{87CD86}} \color[HTML]{000000} 0.79 & {\cellcolor[HTML]{81CA81}} \color[HTML]{000000} 0.86 & {\cellcolor[HTML]{81CA81}} \color[HTML]{000000} 0.86 \\
\bfseries Greek & {\cellcolor[HTML]{8DD08A}} \color[HTML]{000000} 0.73 & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{97D492}} \color[HTML

/tmp/ipykernel_634/4165235407.py:9: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))


In [76]:
#social media EN+ES test data - per platform

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
 for model,data in detector.items():
  temp_data = data[(data.split == 'test') & (data.domain == 'social_media') & (data.language.isin(['en', 'es']))]
 if ('cross-platform' not in model): continue
 temp_data['source'] = temp_data['source'].str.replace('multisocial_', '')
 temp_data['prediction_probs']= temp_data['prediction_probs'].astype(float)
 temp = temp_data
 llm = '{all}'
 labels = [label2id[x] for x in temp['label']]
 fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
 auc_dict[f"{model} {llm}"] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
 for llm in tqdm(temp_data.source.unique(), total=len(temp_data.multi_label.unique())):
  temp = temp_data[(temp_data.source.str.contains(f'{llm}'))]
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[f"{model} {llm}"] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': extract_platform(model.split(' ')[0]).split('_')[0], 'Train Platform': extract_platform(model.split(' ')[0]).split('_')[1], 'Platform': model.split(' ')[1], 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
#results_all.set_index(['Detector', 'Platform', 'Language']).unstack().style.format(precision=2).highlight_max(props='font-weight: bold;', axis=0).background_gradient(vmin=0.5, axis=None)
#results_all['Category'] = results_all['Detector'].map(to_category)
results_all['Detector'] = results_all['Detector'].apply(rename_detector)
results_all['Platform'] = results_all['Platform'].map(rename_platforms)
results_platform = results_all.copy()

 43%|████████████████████████████████████                                               | 36/83 [00:03<00:04,  9.95it/s]/tmp/ipykernel_634/1233827493.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['source'] = temp_data['source'].str.replace('multisocial_', '')
/tmp/ipykernel_634/1233827493.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['prediction_probs']= temp_data['prediction_probs'].astype(float)

 45%|█████████████████████████████████████                                    

In [77]:
#temp = results_all.set_index(['Detector', 'Train Platform', 'Platform']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
temp = results_all[~results_all['Train Platform'].str.contains('x5')].set_index(['Detector', 'Train Platform', 'Platform']).unstack().style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))

\begin{tabular}{llrrrrrr}
 &  & \multicolumn{6}{r}{\bfseries AUC ROC} \\
 & Platform & \bfseries Discord & \bfseries Gab & \bfseries Telegram & \bfseries Twitter & \bfseries WhatsApp & \bfseries {all} \\
Detector & Train Platform &  &  &  &  &  &  \\
\hline
\multirow[c]{6}{*}{\rotatebox{90}{\parbox{2cm}{\bfseries Llama-3-8b}}} & \bfseries Discord & {\cellcolor[HTML]{79ABD0}} \color[HTML]{000000} 0.99 & {\cellcolor[HTML]{B7C5DF}} \color[HTML]{000000} 0.83 & {\cellcolor[HTML]{B7C5DF}} \color[HTML]{000000} 0.82 & {\cellcolor[HTML]{D8D7E9}} \color[HTML]{000000} 0.72 & {\cellcolor[HTML]{ACC0DD}} \color[HTML]{000000} 0.86 & {\cellcolor[HTML]{B4C4DF}} \color[HTML]{000000} 0.83 \\
\bfseries  & \bfseries Gab & {\cellcolor[HTML]{86B0D3}} \color[HTML]{000000} 0.95 & {\cellcolor[HTML]{86B0D3}} \color[HTML]{000000} 0.96 & {\cellcolor[HTML]{8EB3D5}} \color[HTML]{000000} 0.93 & {\cellcolor[HTML]{81AED2}} \color[HTML]{000000} 0.97 & {\cellcolor[HTML]{9CB9D9}} \color[HTML]{000000} 0.90 & {\cellcolor[HT

/tmp/ipykernel_634/1996549063.py:4: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('\\multirow[c]{6}{*}{', '\\multirow[c]{6}{*}{\\rotatebox{90}{\\parbox{2cm}{').replace('} &', '}}} &').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A'))


In [78]:
temp = results_all[results_all['Train Platform'] != 'all'].set_index(['Detector', 'Train Platform', 'Platform']).unstack()[[('AUC ROC', x) for x in ['Discord', 'WhatsApp', 'Telegram', 'Gab', 'Twitter']]].corr().reset_index().drop(columns=['level_0']).set_index('Platform').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))

/tmp/ipykernel_634/4010325801.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  temp = results_all[results_all['Train Platform'] != 'all'].set_index(['Detector', 'Train Platform', 'Platform']).unstack()[[('AUC ROC', x) for x in ['Discord', 'WhatsApp', 'Telegram', 'Gab', 'Twitter']]].corr().reset_index().drop(columns=['level_0']).set_index('Platform').style.format(precision=2).background_gradient(cmap='Greens', vmin=-1, vmax=3, text_color_threshold=0, axis=None).applymap(lambda x: 'background-color: white;' if str(x)=='nan' else '')
/tmp/ipykernel_634/4010325801.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  temp = results_all[results_all['Train Platform'] != 'all'].set_index(['Detector', 'Train Platform', 'Platform']).unstack()[[('AUC ROC', x) for x in ['Discord', 'WhatsApp', 'Telegram', 'Gab', 'Twitter']]].corr().reset_index().drop(columns=['level_0']).set_index('Platform').style.f

\begin{tabular}{lrrrrr}
 & \multicolumn{5}{r}{\bfseries AUC ROC} \\
Platform & \bfseries Discord & \bfseries WhatsApp & \bfseries Telegram & \bfseries Gab & \bfseries Twitter \\
Platform &  &  &  &  &  \\
\bfseries Discord & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{B8E3B2}} \color[HTML]{000000} 0.19 & {\cellcolor[HTML]{C2E7BB}} \color[HTML]{000000} 0.07 & {\cellcolor[HTML]{D0EDCA}} \color[HTML]{000000} -0.14 & {\cellcolor[HTML]{DCF2D7}} \color[HTML]{000000} -0.34 \\
\bfseries WhatsApp & {\cellcolor[HTML]{B8E3B2}} \color[HTML]{000000} 0.19 & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HTML]{86CC85}} \color[HTML]{000000} 0.80 & {\cellcolor[HTML]{95D391}} \color[HTML]{000000} 0.64 & {\cellcolor[HTML]{99D595}} \color[HTML]{000000} 0.58 \\
\bfseries Telegram & {\cellcolor[HTML]{C2E7BB}} \color[HTML]{000000} 0.07 & {\cellcolor[HTML]{86CC85}} \color[HTML]{000000} 0.80 & {\cellcolor[HTML]{73C476}} \color[HTML]{000000} 1.00 & {\cellcolor[HT

/tmp/ipykernel_634/4010325801.py:3: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True))


### Statistics

In [79]:
#T-test for all combinations of train languages
temp = results_language.copy()
temp = pd.DataFrame(multidomain[(multidomain.domain=='social_media') & multidomain.source.str.contains('telegram')][['language']].value_counts())
temp = temp.reset_index()
temp = temp.groupby(['language']).sum().unstack().fillna(0).astype(int) > 2000 #if there are enough samples (~ 250 of each generator)
temp.index = temp.index.droplevel(0)
temp['{all}'] = True
enough_platform_language_samples = temp

temp = results_language.copy()
temp = temp[~temp.Language.isin(['ga', 'gd', 'sk', 'sl'])]#exclude languages with not enough samples
temp = temp[~temp['Train Language'].isin(['{en-es-ru-x3}'])]#exclude training with 3x more samples
languages = temp.Language.unique()
temp = temp.pivot(index=['Train Language', 'Detector'], columns='Language', values='AUC ROC').reset_index()

res_df = pd.DataFrame()
for lang in languages:#[languages != '{all}']:
 for (src, trg) in itertools.combinations_with_replacement(temp['Train Language'].unique(), 2):
   if src == trg: continue
   try:
    res = pg.ttest(temp[(temp["Train Language"] == trg)][lang], temp[(temp["Train Language"] == src)][lang], paired=True)
    #if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest language: {lang}, ({trg}, {src})")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))

    res = stats.ttest_rel(temp[temp["Train Language"] == trg][lang], temp[temp["Train Language"] == src][lang])
    res_df = pd.concat([res_df, pd.DataFrame({'Test Language':lang, 'Train Language':trg, '0.95 Confidence Intrval Min':stats.bayes_mvs(temp[temp["Train Language"] == trg][lang], alpha=0.95)[0].minmax[0], '0.95 Confidence Intrval Max':stats.bayes_mvs(temp[temp["Train Language"] == trg][lang], alpha=0.95)[0].minmax[1]}, index=[0])], ignore_index=True)
    res_df = pd.concat([res_df, pd.DataFrame({'Test Language':lang, 'Train Language':src, '0.95 Confidence Intrval Min':stats.bayes_mvs(temp[temp["Train Language"] == src][lang], alpha=0.95)[0].minmax[0], '0.95 Confidence Intrval Max':stats.bayes_mvs(temp[temp["Train Language"] == src][lang], alpha=0.95)[0].minmax[1]}, index=[0])], ignore_index=True)
   except:
    pass
res_df = res_df.drop_duplicates().sort_values(by=['Test Language', 'Train Language']).reset_index(drop=True)

temp_mean = temp.groupby('Train Language').mean(numeric_only=True)
#sort_key = {'en': 0, 'es': 1, 'ru': 2, 'all': 3, 'en3': 4}
#temp_mean = temp_mean.sort_index(key=lambda x: x.map(sort_key)).loc[:'all',:]
res_df['ci'] = (res_df['0.95 Confidence Intrval Max'] - res_df['0.95 Confidence Intrval Min']) / 2
res_df.drop(columns=['0.95 Confidence Intrval Max', '0.95 Confidence Intrval Min'], inplace=True)
temp_ci = res_df.set_index(['Train Language', 'Test Language']).stack().unstack(level=1).reset_index().set_index('Train Language').drop(columns=['level_1'])#.sort_index(key=lambda x: x.map(sort_key))
temp_mean_bak = temp_mean.copy()

#bg_text_color_threshold = 0.1
temp_mean = temp_mean_bak.copy()
temp = temp_mean.copy()
for col in temp_mean.columns:
  temp[col] = [f"{str('%.2f' % (x))} (±{str('%.2f' % (y))})" if y!='n.s.' else f"{str('%.2f' % (x))} (n.s.)" for x,y in zip(temp_mean[col], temp_ci[col])]

#temp = pd.concat([temp, pd.DataFrame({'Language Average': temp_mean.mean(axis=1)})], axis=1)
#temp_mean = pd.concat([temp_mean, pd.DataFrame({'Language Average': temp_mean.mean(axis=1)})], axis=1)

#temp_mean.rename(index=rename_obfuscators, inplace=True)
#temp.rename(index=rename_obfuscators, inplace=True)

temp[[x for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
temp_mean[[x for x in enough_platform_language_samples[~enough_platform_language_samples].index.tolist()]] = 'nan'
#temp = temp.style.format(precision=2).background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, axis=None)
#print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\multirow[c]{4}{*}{', '\\multirow[c]{4}{*}{\\rotatebox{90}{\\parbox{1.8cm}{').replace('} &', '}}} &').replace('{en-es-ru}}}', '{en-es-ru}').replace('mDeBERTa-v3-base', 'mDeBER\-Ta-v3-base'))
temp = temp.sort_values(by=['Language'], axis=1)
temp = temp.style.background_gradient(vmin=0.5, vmax=1.5, text_color_threshold=0, gmap=temp_mean, axis=None).format({'Language Average': '{:.2%}'.format}, precision=2)
display(temp)
print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}').replace('\\color[HTML]{000000} ', '\\textcolor{black}{').replace(') & {\\cellcolor', ')} & {\\cellcolor').replace(') & N/A', ')} & N/A').replace(' \\', '} \\').replace('&}', '&').replace('}}}', '}}'))


Test language: {all}, (es, en)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.638733,6,two-sided,0.546590,[-0.02 0.01],0.086229,0.417,0.054333



Test language: {all}, (ru, en)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.325776,6,two-sided,0.233153,[-0.08 0.02],0.393308,0.675,0.142849



Test language: {all}, ({en-es-ru}, en)


...


Test language: en, ({en-es-ru}, ru)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.637307,6,two-sided,0.010870,[0.03 0.13],1.862422,6.297,0.981721


Language,ar,bg,ca,cs,de,el,en,es,et,ga,gd,hr,hu,nl,pl,pt,ro,ru,sk,sl,uk,zh,{all}
Train Language,,,,,,,,,,,,,,,,,,,,,,,
en,0.81 (±0.05),0.90 (±0.08),0.78 (±0.03),0.91 (±0.05),0.88 (±0.02),0.90 (±0.03),0.96 (±0.01),0.87 (±0.06),0.92 (±0.03),nan,nan,0.94 (±0.03),0.97 (±0.03),0.84 (±0.02),0.89 (±0.05),0.92 (±0.02),0.94 (±0.02),0.87 (±0.05),nan,nan,0.81 (±0.05),0.73 (±0.14),0.87 (±0.04)
es,0.82 (±0.05),0.89 (±0.08),0.85 (±0.03),0.89 (±0.06),0.89 (±0.03),0.87 (±0.06),0.90 (±0.04),0.94 (±0.01),0.90 (±0.05),nan,nan,0.92 (±0.05),0.95 (±0.04),0.84 (±0.03),0.88 (±0.06),0.93 (±0.02),0.93 (±0.03),0.88 (±0.04),nan,nan,0.82 (±0.05),0.73 (±0.14),0.86 (±0.05)
ru,0.81 (±0.10),0.93 (±0.05),0.76 (±0.10),0.87 (±0.08),0.84 (±0.04),0.88 (±0.06),0.87 (±0.06),0.82 (±0.11),0.88 (±0.08),nan,nan,0.89 (±0.07),0.91 (±0.07),0.79 (±0.06),0.87 (±0.07),0.86 (±0.07),0.88 (±0.07),0.94 (±0.02),nan,nan,0.89 (±0.04),0.73 (±0.17),0.84 (±0.08)
{en-es-ru},0.89 (±0.03),0.94 (±0.04),0.86 (±0.03),0.93 (±0.03),0.91 (±0.03),0.93 (±0.02),0.95 (±0.01),0.93 (±0.02),0.93 (±0.03),nan,nan,0.94 (±0.04),0.97 (±0.02),0.86 (±0.03),0.91 (±0.04),0.94 (±0.01),0.95 (±0.02),0.93 (±0.03),nan,nan,0.89 (±0.03),0.86 (±0.08),0.91 (±0.03)


\begin{tabular}{llllllllllllllllllllllll}
Language & \bfseries ar & \bfseries bg & \bfseries ca & \bfseries cs & \bfseries de & \bfseries el & \bfseries en & \bfseries es & \bfseries et & \bfseries ga & \bfseries gd & \bfseries hr & \bfseries hu & \bfseries nl & \bfseries pl & \bfseries pt & \bfseries ro & \bfseries ru & \bfseries sk & \bfseries sl & \bfseries uk & \bfseries zh & \bfseries {all}} \\
Train Language &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  & } \\
\bfseries en & {\cellcolor[HTML]{BCC7E1}} \textcolor{black}{0.81 (±0.05)} & {\cellcolor[HTML]{9CB9D9}} \textcolor{black}{0.90 (±0.08)} & {\cellcolor[HTML]{C6CCE3}} \textcolor{black}{0.78 (±0.03)} & {\cellcolor[HTML]{99B8D8}} \textcolor{black}{0.91 (±0.05)} & {\cellcolor[HTML]{A5BDDB}} \textcolor{black}{0.88 (±0.02)} & {\cellcolor[HTML]{9AB8D8}} \textcolor{black}{0.90 (±0.03)} & {\cellcolor[HTML]{84B0D3}} \textcolor{black}{0.96 (±0.01)} & {\cellcolor[HTML]{A8BEDC}} \textcolor{black}{0.87 (±0.06)} & {\cell

/tmp/ipykernel_634/757126828.py:60: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(temp.applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).to_latex(convert_css=True).replace('\\multirow', '\\hline\n\\multirow').replace('{\\cellcolor[HTML]{000000}} \\color[HTML]{000000} nan', 'N/A').replace('nan', 'N/A').replace('\\bfseries {en-es-ru}', '\hline\n\\bfseries {en-es-ru}').replace('\\color[HTML]{000000} ', '\\textcolor{black}{').replace(') & {\\cellcolor', ')} & {\\cellcolor').replace(') & N/A', ')} & N/A').replace(' \\', '} \\').replace('&}', '&').replace('}}}', '}}'))


In [80]:
temp_mean.astype(float).style.highlight_max(props='font-weight: bold;', axis=0)

Language,ar,bg,ca,cs,de,el,en,es,et,hr,hu,nl,pl,pt,ro,ru,uk,zh,{all},ga,gd,sk,sl
Train Language,,,,,,,,,,,,,,,,,,,,,,,
en,0.809295,0.899167,0.779832,0.909984,0.878790,0.904746,0.960253,0.870765,0.917832,0.938125,0.966266,0.839966,0.890541,0.920229,0.941010,0.872232,0.814642,0.727139,0.867748,nan,nan,nan,nan
es,0.824158,0.888744,0.845237,0.885706,0.893328,0.871900,0.901816,0.938009,0.898208,0.915434,0.949759,0.837107,0.878015,0.925194,0.931187,0.884376,0.823268,0.733953,0.863598,nan,nan,nan,nan
ru,0.805230,0.930442,0.755730,0.872877,0.841072,0.881664,0.874709,0.821996,0.877180,0.892345,0.911441,0.794452,0.872687,0.861349,0.882712,0.939087,0.890963,0.731219,0.840632,nan,nan,nan,nan
{en-es-ru},0.889312,0.942544,0.862008,0.934025,0.909034,0.931246,0.954641,0.932067,0.932230,0.941341,0.974758,0.859795,0.910343,0.938163,0.948052,0.933666,0.891547,0.861325,0.911614,nan,nan,nan,nan


In [81]:
#statistical significance between language-differently finetuned detectors
temp = results_language.copy()
temp = temp[~temp.Language.isin(['ga', 'gd', 'sk', 'sl'])]#exclude languages with not enough samples
temp = temp[~temp['Train Language'].isin(['{en-es-ru-x3}'])]#exclude training with 3x more samples
languages = temp.Language.unique()
temp = temp.pivot(index=['Train Language', 'Detector'], columns='Language', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])

res_df = pd.DataFrame()
for lang in languages[languages != '{all}']:
 for (src, trg) in itertools.combinations_with_replacement(temp['Train Language'].unique(), 2):
   if src == trg: continue
   try:
    res = pg.ttest(temp[(temp["Train Language"] == trg)][lang], temp[(temp["Train Language"] == src)][lang], paired=True)
    if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest language: {lang}, ({trg}, {src})")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass


Test language: cs, ({en-es-ru}, es)


/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.551696,6,two-sided,0.043391,[0. 0.09],0.884182,2.204,0.501279



Test language: cs, ({en-es-ru}, ru)


/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


...

/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.401772,6,two-sided,0.014464,[0.01 0.09],1.674619,5.055,0.955051



Test language: en, ({en-es-ru}, ru)


/tmp/ipykernel_634/2093459691.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.637307,6,two-sided,0.010870,[0.03 0.13],1.862422,6.297,0.981721


In [82]:
#statistical significance between monolingually and multilingually finetuned detectors
temp = results_language.copy()
temp = temp[~temp.Language.isin(['ga', 'gd', 'sk', 'sl'])]#exclude languages with not enough samples
temp = temp[~temp['Train Language'].isin(['{en-es-ru-x3}'])]#exclude training with 3x more samples
languages = temp.Language.unique()
temp = temp.pivot(index=['Train Language', 'Detector'], columns='Language', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])

res_df = pd.DataFrame()
#for (src, trg) in itertools.combinations_with_replacement(temp['Train Language'].unique(), 2):
#  if src == trg: continue
for lang in languages[languages != '{all}']:
   try:
    res = pg.ttest(temp[(temp["Train Language"] == '{en-es-ru}')][lang], temp[(temp["Train Language"] != '{en-es-ru}')][lang], paired=True)
    #res = pg.ttest(temp[(temp["Train Language"] == '{en-es-ru}')][lang], temp[(temp["Train Language"] != '{en-es-ru}')].groupby('Detector')[lang].mean(), paired=True)
    if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest language: {lang}")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass

/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")



Test language: cs


/tmp/ipykernel_634/2830034179.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.252330,21.336179,two-sided,0.034949,[0. 0.09],0.714576,2.177,0.351051



Test language: ar


...


Test language: zh


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.838212,19.282530,two-sided,0.010407,[0.03 0.23],0.936575,5.559,0.542394



Test language: en


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")
/tmp/ipykernel_634/2830034179.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")
/tmp/ipykernel_634/2830034179.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.413493,22.678633,two-sided,0.002414,[0.02 0.07],0.877190,16.111,0.490274


In [83]:
#statistical significance between monolingually and multilingually finetuned detectors autoregressive-only
temp = results_language.copy()
temp = temp[~temp.Language.isin(['ga', 'gd', 'sk', 'sl'])]#exclude languages with not enough samples
temp = temp[~temp['Train Language'].isin(['{en-es-ru-x3}'])]#exclude training with 3x more samples
temp = temp[temp.Detector.isin(autoregressive)]#exclude not autoregressive models
languages = temp.Language.unique()
temp = temp.pivot(index=['Train Language', 'Detector'], columns='Language', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])

res_df = pd.DataFrame()
#for (src, trg) in itertools.combinations_with_replacement(temp['Train Language'].unique(), 2):
#  if src == trg: continue
for lang in languages[languages != '{all}']:
   try:
    res = pg.ttest(temp[(temp["Train Language"] == '{en-es-ru}')][lang], temp[(temp["Train Language"] != '{en-es-ru}')][lang], paired=True)
    #res = pg.ttest(temp[(temp["Train Language"] == '{en-es-ru}')][lang], temp[(temp["Train Language"] != '{en-es-ru}')].groupby('Detector')[lang].mean(), paired=True)
    if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest language: {lang}")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass

/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")



Test language: cs


/tmp/ipykernel_634/779534088.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.843354,8.174722,two-sided,0.004727,[0.03 0.12],1.787533,17.195,0.820757


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")
/tmp/ipykernel_634/779534088.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


...

/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")
/tmp/ipykernel_634/779534088.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.459010,9.126779,two-sided,0.035870,[0. 0.1],1.097308,2.543,0.424839



Test language: en


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")
/tmp/ipykernel_634/779534088.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")
/tmp/ipykernel_634/779534088.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.686266,12.589695,two-sided,0.002883,[0.03 0.11],1.244836,13.639,0.518943


In [84]:
#statistical significance between platform-differently finetuned detectors
temp = results_platform.copy()
temp = temp[~temp['Train Platform'].str.contains('x5')]#exclude training with 5x more samples
platforms = temp.Platform.unique()
temp = temp.pivot(index=['Train Platform', 'Detector'], columns='Platform', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])
#display(temp)

res_df = pd.DataFrame()
for (src, trg) in itertools.combinations_with_replacement(temp['Train Platform'].unique(), 2):
  if src == trg: continue
  for lang in platforms[platforms != '{all}']:
   try:
    res = pg.ttest(temp[(temp["Train Platform"] == trg)][lang], temp[(temp["Train Platform"] == src)][lang], paired=True)
    if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest platform: {lang}, ({trg}, {src})")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass

/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue



Test platform: Gab, (Gab, Discord)


/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.825367,2,two-sided,0.028226,[0.03 0.19],6.274932,3.576,0.997002


/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ tre

...

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,6.093870,2,two-sided,0.025887,[0.01 0.03],1.834795,3.758,0.419369


/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue



Test platform: WhatsApp, (all, WhatsApp)


/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue
/tmp/ipykernel_634/970638343.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (res['p-val'][0] >= 0.05): continue


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.799327,2,two-sided,0.028470,[-0.04 -0.01],4.667736,3.558,0.960748


In [85]:
#statistical significance between monoplatform and multiplatform finetuning of detectors
temp = results_platform.copy()
temp = temp[~temp['Train Platform'].str.contains('x5')]#exclude training with 5x more samples
platforms = temp.Platform.unique()
temp = temp.pivot(index=['Train Platform', 'Detector'], columns='Platform', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])
#display(temp)

res_df = pd.DataFrame()
#for (src, trg) in itertools.combinations_with_replacement(temp['Train Platform'].unique(), 2):
#  if src == trg: continue
for lang in platforms[platforms != '{all}']:
   try:
    res = pg.ttest(temp[(temp["Train Platform"] == 'all')][lang], temp[(temp["Train Platform"] != 'all')][lang], paired=True)
    #if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest platform: {lang}")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass

/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")



Test platform: Twitter


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.877462,15.985249,two-sided,0.078826,[-0. 0.08],0.549837,1.336,0.129492



Test platform: Telegram


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


...

/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.604018,7.800241,two-sided,0.148343,[-0.01 0.06],0.597453,1.031,0.144262



Test platform: WhatsApp


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.941411,15.978352,two-sided,0.070056,[-0. 0.04],0.567905,1.427,0.134943


In [86]:
#statistical significance between monoplatform and multiplatform finetuning of detectors without mdeberta
temp = results_platform.copy()
temp = temp[~temp['Train Platform'].str.contains('x5')]#exclude training with 5x more samples
temp = temp[~temp.Detector.str.contains('mDeBERTa')]#exclude the small mdeberta model
platforms = temp.Platform.unique()
temp = temp.pivot(index=['Train Platform', 'Detector'], columns='Platform', values='AUC ROC').reset_index()
#temp = pd.concat([temp, temp.loc[temp.Category == 'P', :]])
#display(temp)

res_df = pd.DataFrame()
#for (src, trg) in itertools.combinations_with_replacement(temp['Train Platform'].unique(), 2):
#  if src == trg: continue
for lang in platforms[platforms != '{all}']:
   try:
    res = pg.ttest(temp[(temp["Train Platform"] == 'all')][lang], temp[(temp["Train Platform"] != 'all')][lang], paired=True)
    #if (res['p-val'][0] >= 0.05): continue
    print(f"\nTest platform: {lang}")
    display(res.style.apply(lambda _: np.where(res['p-val'] >= 0.05, 'background-color: yellow', '')))
   except:
    pass


Test platform: Twitter


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.336011,9.961581,two-sided,0.211263,[-0.02 0.1 ],0.464185,0.867,0.084506



Test platform: Telegram


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


...


Test platform: Gab


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.538056,5.411776,two-sided,0.180243,[-0.02 0.08],0.644834,0.996,0.117428



Test platform: WhatsApp


/home/kali/anaconda3/lib/python3.9/site-packages/pingouin/parametric.py:228: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.698750,9.999679,two-sided,0.120211,[-0.01 0.05],0.595596,1.124,0.107324


## News

In [87]:

#news test data - th calibration for macroF1 based on train data
auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    train = data[(data.split == 'train') & (data.domain == 'social_media') & (~data.multi_label.str.contains('gemini'))]
    #print(data[(data.split == 'test') & (data.domain == 'news')].multi_label.unique())
    temp = data[(data.split == 'test') & (data.domain == 'news') & (~data.multi_label.str.contains('lama'))]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)]}#, 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}

  train_labels = [label2id[x] for x in train['label']]
  fpr, tpr, thresholds = roc_curve(train_labels, train['prediction_probs'])
  predictions = [1 if ((x >= thresholds[fpr <= 0.05][-1])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_5'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
  predictions = [1 if ((x >= thresholds[np.argmax(tpr - fpr)])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_o'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': rename_detector(model), 'Category': to_category[model], 'AUC ROC': v['auc'], 'MacroF1@5%FPR': v['cr_5']['macro avg']['f1-score'], 'MacroF1@optim': v['cr_o']['macro avg']['f1-score']}, index=[0])
  results_all = pd.concat([results_all, temp])
results_all = results_all.sort_values(by=['AUC ROC'], ascending=False).reset_index(drop=True)
results_all.index = results_all.index + 1
display(results_all.style.format(na_rep=0, precision=4))
display(results_all.style.apply(highlight_categories, axis=1).hide(['Category', 'MacroF1@optim'], axis=1))
print(results_all.reset_index().style.apply(highlight_categories, axis=1).hide(['Category', 'MacroF1@optim'], axis=1).format(na_rep=0, precision=4).applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).hide(
    axis=0).to_latex(convert_css=True))

100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 2479.16it/s]


,Detector,Category,AUC ROC,MacroF1@5%FPR,MacroF1@optim
1,Llama-3-8b-MultiSocial,F,0.9273,0.7988,0.7999
2,Aya-101-MultiSocial,F,0.9262,0.8008,0.8010
3,mDeBERTa-v3-base-MultiSocial,F,0.9025,0.7512,0.7748
4,Mistral-7b-v0.1-MultiSocial,F,0.8988,0.7937,0.7892
5,XLM-RoBERTa-large-MultiSocial,F,0.8309,0.7306,0.6840
6,Binoculars,S,0.8303,0.4041,0.5539
7,Fast-Detect-GPT,S,0.8104,0.6361,0.5961
8,Falcon-rw-1b-MultiSocial,F,0.7592,0.6394,0.6599
9,BLOOMZ-3b-MultiSocial,F,0.7071,0.5731,0.5959
10,LLM-Deviation,S,0.6568,0.3568,0.4649


,Detector,AUC ROC,MacroF1@5%FPR
1,Llama-3-8b-MultiSocial,0.927322,0.798831
2,Aya-101-MultiSocial,0.926166,0.800815
3,mDeBERTa-v3-base-MultiSocial,0.902500,0.751232
4,Mistral-7b-v0.1-MultiSocial,0.898766,0.793683
5,XLM-RoBERTa-large-MultiSocial,0.830872,0.730627
6,Binoculars,0.830301,0.404108
7,Fast-Detect-GPT,0.810407,0.636138
8,Falcon-rw-1b-MultiSocial,0.759205,0.639355
9,BLOOMZ-3b-MultiSocial,0.707063,0.573052
10,LLM-Deviation,0.656790,0.356787


\begin{tabular}{rlrr}
\bfseries index & \bfseries Detector & \bfseries AUC ROC & \bfseries MacroF1@5%FPR \\
{\cellcolor[HTML]{B6D7A8}} 1 & {\cellcolor[HTML]{B6D7A8}} Llama-3-8b-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9273 & {\cellcolor[HTML]{B6D7A8}} 0.7988 \\
{\cellcolor[HTML]{B6D7A8}} 2 & {\cellcolor[HTML]{B6D7A8}} Aya-101-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9262 & {\cellcolor[HTML]{B6D7A8}} 0.8008 \\
{\cellcolor[HTML]{B6D7A8}} 3 & {\cellcolor[HTML]{B6D7A8}} mDeBERTa-v3-base-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.9025 & {\cellcolor[HTML]{B6D7A8}} 0.7512 \\
{\cellcolor[HTML]{B6D7A8}} 4 & {\cellcolor[HTML]{B6D7A8}} Mistral-7b-v0.1-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.8988 & {\cellcolor[HTML]{B6D7A8}} 0.7937 \\
{\cellcolor[HTML]{B6D7A8}} 5 & {\cellcolor[HTML]{B6D7A8}} XLM-RoBERTa-large-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.8309 & {\cellcolor[HTML]{B6D7A8}} 0.7306 \\
{\cellcolor[HTML]{F9CB9C}} 6 & {\cellcolor[HTML]{F9CB9C}} Binoculars & {\cellcolor[HTML]{F9CB9C}}

/tmp/ipykernel_634/2479829085.py:29: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(results_all.reset_index().style.apply(highlight_categories, axis=1).hide(['Category', 'MacroF1@optim'], axis=1).format(na_rep=0, precision=4).applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).hide(


In [88]:

#(OOD) news test data of Llama-2-70b generator - th calibration for macroF1 based on train data
auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    train = data[(data.split == 'train') & (data.domain == 'social_media') & (~data.multi_label.str.contains('gemini'))]
    #print(data[(data.split == 'test') & (data.domain == 'news')].multi_label.unique())
    temp = data[(data.split == 'test') & (data.domain == 'news') & (data.multi_label.str.contains('lama') | data.multi_label.str.contains('human'))]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)]}#, 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}

  train_labels = [label2id[x] for x in train['label']]
  fpr, tpr, thresholds = roc_curve(train_labels, train['prediction_probs'])
  predictions = [1 if ((x >= thresholds[fpr <= 0.05][-1])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_5'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
  predictions = [1 if ((x >= thresholds[np.argmax(tpr - fpr)])) else 0 for x in temp['prediction_probs']]
  auc_dict[model]['cr_o'] = report_todict(*report_np(np.array(labels), np.array(predictions), 2))
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': rename_detector(model), 'Category': to_category[model], 'AUC ROC': v['auc'], 'MacroF1@5%FPR': v['cr_5']['macro avg']['f1-score'], 'MacroF1@optim': v['cr_o']['macro avg']['f1-score']}, index=[0])
  results_all = pd.concat([results_all, temp])
results_all = results_all.sort_values(by=['AUC ROC'], ascending=False).reset_index(drop=True)
results_all.index = results_all.index + 1
display(results_all.style.format(na_rep=0, precision=4))
display(results_all.style.apply(highlight_categories, axis=1).hide(['Category', 'MacroF1@optim'], axis=1))
print(results_all.reset_index().style.apply(highlight_categories, axis=1).hide(['Category', 'MacroF1@optim'], axis=1).format(na_rep=0, precision=4).applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).hide(
    axis=0).to_latex(convert_css=True))

100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1860.24it/s]


,Detector,Category,AUC ROC,MacroF1@5%FPR,MacroF1@optim
1,Fast-Detect-GPT,S,0.9238,0.8471,0.5841
2,Binoculars,S,0.9048,0.7568,0.4473
3,mDeBERTa-v3-base-MultiSocial,F,0.8871,0.8011,0.7916
4,Mistral-7b-v0.1-MultiSocial,F,0.8614,0.7673,0.7732
5,Aya-101-MultiSocial,F,0.8574,0.7556,0.7562
6,Llama-3-8b-MultiSocial,F,0.8549,0.7352,0.7451
7,XLM-RoBERTa-large-MultiSocial,F,0.7928,0.7108,0.6032
8,Falcon-rw-1b-MultiSocial,F,0.7710,0.6912,0.6707
9,DetectLLM-LRR,S,0.7559,0.7121,0.3320
10,LLM-Deviation,S,0.7257,0.5931,0.3319


,Detector,AUC ROC,MacroF1@5%FPR
1,Fast-Detect-GPT,0.923793,0.847065
2,Binoculars,0.904783,0.756778
3,mDeBERTa-v3-base-MultiSocial,0.887064,0.801123
4,Mistral-7b-v0.1-MultiSocial,0.861438,0.767348
5,Aya-101-MultiSocial,0.857379,0.755611
6,Llama-3-8b-MultiSocial,0.854890,0.735153
7,XLM-RoBERTa-large-MultiSocial,0.792760,0.710824
8,Falcon-rw-1b-MultiSocial,0.770981,0.691165
9,DetectLLM-LRR,0.755920,0.712063
10,LLM-Deviation,0.725709,0.593085


\begin{tabular}{rlrr}
\bfseries index & \bfseries Detector & \bfseries AUC ROC & \bfseries MacroF1@5%FPR \\
{\cellcolor[HTML]{F9CB9C}} 1 & {\cellcolor[HTML]{F9CB9C}} Fast-Detect-GPT & {\cellcolor[HTML]{F9CB9C}} 0.9238 & {\cellcolor[HTML]{F9CB9C}} 0.8471 \\
{\cellcolor[HTML]{F9CB9C}} 2 & {\cellcolor[HTML]{F9CB9C}} Binoculars & {\cellcolor[HTML]{F9CB9C}} 0.9048 & {\cellcolor[HTML]{F9CB9C}} 0.7568 \\
{\cellcolor[HTML]{B6D7A8}} 3 & {\cellcolor[HTML]{B6D7A8}} mDeBERTa-v3-base-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.8871 & {\cellcolor[HTML]{B6D7A8}} 0.8011 \\
{\cellcolor[HTML]{B6D7A8}} 4 & {\cellcolor[HTML]{B6D7A8}} Mistral-7b-v0.1-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.8614 & {\cellcolor[HTML]{B6D7A8}} 0.7673 \\
{\cellcolor[HTML]{B6D7A8}} 5 & {\cellcolor[HTML]{B6D7A8}} Aya-101-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.8574 & {\cellcolor[HTML]{B6D7A8}} 0.7556 \\
{\cellcolor[HTML]{B6D7A8}} 6 & {\cellcolor[HTML]{B6D7A8}} Llama-3-8b-MultiSocial & {\cellcolor[HTML]{B6D7A8}} 0.8549 & {\ce

/tmp/ipykernel_634/2866092777.py:29: FutureWarning: Styler.applymap_index has been deprecated. Use Styler.map_index instead.
  print(results_all.reset_index().style.apply(highlight_categories, axis=1).hide(['Category', 'MacroF1@optim'], axis=1).format(na_rep=0, precision=4).applymap_index(lambda v: "font-weight: bold;", axis=0).applymap_index(lambda v: "font-weight: bold;", axis=1).hide(


In [89]:
#news test data

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    temp = data[(data.split == 'test') & (data.domain == 'news') & (~data.multi_label.str.contains('lama'))]
    #temp = data[(data.split == 'test') & (data.domain == 'news') & (data.multi_label.str.contains('lama') | data.multi_label.str.contains('human'))]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  temp['prediction_probs']= temp['prediction_probs'].astype(float)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[model][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model, 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model, 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
results_all.set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).style.format(precision=2).apply(lambda x: ["background-color: lightyellow" if v >= 0.6 else "" for v in x], axis = 0, subset='AUC ROC').highlight_max(props='font-weight: bold;', axis=0)

  0%|                                                                                            | 0/83 [00:00<?, ?it/s]/tmp/ipykernel_634/3409598150.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['prediction_probs']= temp['prediction_probs'].astype(float)
  1%|█                                                                                   | 1/83 [00:00<00:30,  2.68it/s]/tmp/ipykernel_634/3409598150.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['prediction_probs']= temp['predictio

In [90]:
#news test data - OOD

auc_dict = {}
for detector in tqdm(test_results, total=len(test_results)):
  for model,data in detector.items():
    #temp = data[(data.split == 'test') & (data.domain == 'news') & (~data.multi_label.str.contains('lama'))]
    temp = data[(data.split == 'test') & (data.domain == 'news') & (data.multi_label.str.contains('lama') | data.multi_label.str.contains('human'))]
  if (to_category[model] == 'F') and ('social-media' not in model): continue
  #if (temp['prediction_probs'].min() >= 0.5) and (temp['prediction_probs'].max() <= 1.0):
    #temp.loc[temp.predictions == 'human', 'prediction_probs'] = 1 - temp['prediction_probs'] #adjust human prediction prob to be in range <0, 0.5)
  temp['prediction_probs']= temp['prediction_probs'].astype(float)
  labels = [label2id[x] for x in temp['label']]
  fpr, tpr, thresholds = roc_curve(labels, temp['prediction_probs'])
  auc_dict[model] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
  for test_language in temp.language.unique():
    temp2 = temp[temp.language == test_language].reset_index(drop=True)
    labels = [label2id[x] for x in temp2['label']]
    fpr, tpr, thresholds = roc_curve(labels, temp2['prediction_probs'])
    auc_dict[model][test_language] = {'auc':  auc_roc_reliable(fpr, tpr, labels), 'th_optim': thresholds[np.argmax(tpr - fpr)], 'tpr_1%fpr': tpr[fpr <= 0.01][-1], 'tpr_5%fpr': tpr[fpr <= 0.05][-1], 'th_1%fpr': thresholds[fpr <= 0.01][-1], 'th_3%fpr': thresholds[fpr <= 0.03][-1], 'th_5%fpr': thresholds[fpr <= 0.05][-1], 'th_10%fpr': thresholds[fpr <= 0.10][-1], 'th_15%fpr': thresholds[fpr <= 0.15][-1], 'th_20%fpr': thresholds[fpr <= 0.20][-1], 'th_25%fpr': thresholds[fpr <= 0.25][-1], 'th_30%fpr': thresholds[fpr <= 0.30][-1], 'th_40%fpr': thresholds[fpr <= 0.40][-1], 'th_50%fpr': thresholds[fpr <= 0.50][-1]}
results_all = pd.DataFrame()
for model, v in tqdm(auc_dict.items(), total=len(auc_dict)):
  temp = pd.DataFrame({'Detector': model, 'Language': '{all}', 'AUC ROC': v['auc']}, index=[0])
  results_all = pd.concat([results_all, temp])
  for test_language,val in v.items():
    if (test_language == 'auc') or ('_' in test_language): continue
    temp = pd.DataFrame({'Detector': model, 'Language': test_language, 'AUC ROC': val['auc']}, index=[0])
    results_all = pd.concat([results_all, temp])
results_all.set_index(['Detector', 'Language']).unstack().sort_values(by=('AUC ROC','{all}'), ascending=False).style.format(precision=2).apply(lambda x: ["background-color: lightyellow" if v >= 0.6 else "" for v in x], axis = 0, subset='AUC ROC').highlight_max(props='font-weight: bold;', axis=0)

  0%|                                                                                            | 0/83 [00:00<?, ?it/s]/tmp/ipykernel_634/682490615.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['prediction_probs']= temp['prediction_probs'].astype(float)
  1%|█                                                                                   | 1/83 [00:00<00:31,  2.64it/s]/tmp/ipykernel_634/682490615.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['prediction_probs']= temp['prediction_